# Developping a Borrower Scoring Algorithm

Last updated : September 25th, 2022

## Introduction

During this project, I will use a dataset provided by a consumer finance companies to develop a machine learning algorithm that will predict if the borrower will have payment difficulties or not.

## 1. Data Loading and Filtering

First we will load the necessary packages and dataset and then we will carry on with the Cleaning and Analysis.

### 1.1 Loading our packages

We will import the necessary packages to run this project: matplotlib, numpy, pandas, seaborn.
Since I am running the project on Windows, I will also use sklearnex to increase the speed of sklearn.

In [17]:
#Importing packages
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
#Setting large figure size for Seaborn
sns.set(rc={'figure.figsize':(11.7,8.27),"font.size":20,"axes.titlesize":20,"axes.labelsize":18})

#Importing Intel extension for sklearn to improve speed
from sklearnex import patch_sklearn
patch_sklearn()

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


### 1.2 Loading the dataset

We will now load the dataset

In [18]:
# app_test = pd.read_csv("Data/application_test.csv", sep=",")
# app = pd.read_csv("Data/application_train.csv", sep=",")

# app.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


### 1.3 Feature Filtering

We will begin by removing features that have more than 50% na values :

In [19]:
# Increasing maximum number of info rows 
pd.options.display.max_info_columns = 130

# #First we will define a function that drops columns that are null in more than x% of our database
# def drop_na_columns(df: pd.DataFrame, percent: float):
#     n = len(df)
#     cutoff = n*percent/100
#     for c in df.columns:
#         if len(df[c].dropna()) < cutoff:
#             df.drop(columns={c}, inplace=True)

# #Dropping columns with less than 50% complete fields
# drop_na_columns(app, 50)

# len(app.columns)

# app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Data columns (total 81 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   SK_ID_CURR                    307511 non-null  int64  
 1   TARGET                        307511 non-null  int64  
 2   NAME_CONTRACT_TYPE            307511 non-null  object 
 3   CODE_GENDER                   307511 non-null  object 
 4   FLAG_OWN_CAR                  307511 non-null  object 
 5   FLAG_OWN_REALTY               307511 non-null  object 
 6   CNT_CHILDREN                  307511 non-null  int64  
 7   AMT_INCOME_TOTAL              307511 non-null  float64
 8   AMT_CREDIT                    307511 non-null  float64
 9   AMT_ANNUITY                   307499 non-null  float64
 10  AMT_GOODS_PRICE               307233 non-null  float64
 11  NAME_TYPE_SUITE               306219 non-null  object 
 12  NAME_INCOME_TYPE              307511 non-nul

In [20]:
# #Counting the number of target vs not target variables:
# app["TARGET"].value_counts(normalize=True)

# #We have a significant difference in the number of data for both cases

0    0.919271
1    0.080729
Name: TARGET, dtype: float64

## 2. Data Preparation

We will now clean our dataset.

### 2.1 Cleaning categorical variables

We will begin the cleaning process by cleaning categorical variables.

In [21]:
# #Looking at unique valeus of categorical variables
# def investigate_categories(df: pd.DataFrame):
#     for c in df.columns:
#         if df[c].dtype == 'object':
#             print("Column",c)
#             print("Unique values: {}".format(df[c].unique()))
#             print("")
#             print("-----------------------------------")
            
# investigate_categories(app)

Column NAME_CONTRACT_TYPE
Unique values: ['Cash loans' 'Revolving loans']

-----------------------------------
Column CODE_GENDER
Unique values: ['M' 'F' 'XNA']

-----------------------------------
Column FLAG_OWN_CAR
Unique values: ['N' 'Y']

-----------------------------------
Column FLAG_OWN_REALTY
Unique values: ['Y' 'N']

-----------------------------------
Column NAME_TYPE_SUITE
Unique values: ['Unaccompanied' 'Family' 'Spouse, partner' 'Children' 'Other_A' nan
 'Other_B' 'Group of people']

-----------------------------------
Column NAME_INCOME_TYPE
Unique values: ['Working' 'State servant' 'Commercial associate' 'Pensioner' 'Unemployed'
 'Student' 'Businessman' 'Maternity leave']

-----------------------------------
Column NAME_EDUCATION_TYPE
Unique values: ['Secondary / secondary special' 'Higher education' 'Incomplete higher'
 'Lower secondary' 'Academic degree']

-----------------------------------
Column NAME_FAMILY_STATUS
Unique values: ['Single / not married' 'Married' 'C

In [22]:
# #Investigating "XNA" values in GENDER
# app[app["CODE_GENDER"] == 'XNA']
# #Only 4 rows

# #Let's look at the test data
# app_test[app_test["CODE_GENDER"] == 'XNA']
# #0 row

# #We will replace with the mode
# app["CODE_GENDER"] = app["CODE_GENDER"].fillna(app["CODE_GENDER"].mode())

In [23]:
# #Investigating "XNA" values in ORGANIZATION_TYPE
# app[app["ORGANIZATION_TYPE"] == 'XNA']
# #55374 rows

# app[app["ORGANIZATION_TYPE"] == 'XNA']["TARGET"].value_counts(normalize=True)
# #Significant deviation from the normal percentages, so it is interesting to keep these values

# #They will be encoded during the feature engineering part of the project

0    0.946004
1    0.053996
Name: TARGET, dtype: float64

In [24]:
# #Looking at "nan" values in EMERGENCYSTATE_MODE
# print(len(app[app["EMERGENCYSTATE_MODE"].isna()]))

# app[app["EMERGENCYSTATE_MODE"].isna()]["TARGET"].value_counts(normalize=True)
# #Here it represents about half our dataset, we will create a "NA" variable as well since there is a small deviation from what
# #We would have expected

# app.loc[app["EMERGENCYSTATE_MODE"].isna(),"EMERGENCYSTATE_MODE"] = 'UKN'

145754


In [25]:
# #Looking at "nan" values in OCCUPATION TYPE
# print(len(app[app["OCCUPATION_TYPE"].isna()]))

# app[app["OCCUPATION_TYPE"].isna()]["TARGET"].value_counts(normalize=True)
# #Here it represents about a third of our dataset, we will create a "NA" variable as well since there is a deviation from what
# #we would have expected

# app.loc[app["OCCUPATION_TYPE"].isna(),"OCCUPATION_TYPE"] = 'UKN'

96389


In [26]:
# #Looking at "nan" values in NAME_TYPE_SUITE
# print(len(app[app["NAME_TYPE_SUITE"].isna()]))
# #Only 1292 NA values

# #We will replace these rows by the mode
# app["NAME_TYPE_SUITE"] = app["NAME_TYPE_SUITE"].fillna(app["NAMLE_TYPE_SUITE"].mode())

1292


In [27]:
# #We can see that WEEKDAY_APPR_PROCESS_START is coded as a string

# import time
# #Let's convert it into week day number
# app["WEEKDAY_APPR_PROCESS_START"] = app["WEEKDAY_APPR_PROCESS_START"].apply(lambda x: time.strptime(x, '%A').tm_wday)

In [28]:
# #Verifying that we've dealt with all missing values of categorical variables
# for c in app.columns:
#     if app[c].dtype == 'object':
#         print(app[c].isna().sum().sum())

0
0
0
0
0
0
0
0
0
0
0
0


In [ ]:
import time

#Summarizing preprocessing of categorical variables
def preprocess_cat_vars(df: pd.DataFrame):

    app["CODE_GENDER"] = app["CODE_GENDER"].fillna(app["CODE_GENDER"].mode())

    app.loc[app["EMERGENCYSTATE_MODE"].isna(),"EMERGENCYSTATE_MODE"] = 'UKN'

    app.loc[app["OCCUPATION_TYPE"].isna(),"OCCUPATION_TYPE"] = 'UKN'

    app["NAME_TYPE_SUITE"] = app["NAME_TYPE_SUITE"].fillna(app["NAMLE_TYPE_SUITE"].mode())

    #Let's convert it into week day number
    app["WEEKDAY_APPR_PROCESS_START"] = app["WEEKDAY_APPR_PROCESS_START"].apply(lambda x: time.strptime(x, '%A').tm_wday)
        

We have finished cleaning up categorical variables, now we will look at numeric variables 

### 2.2 Cleaning numeric variables 

In [29]:
#Looking for outliers 

#Increasing the number of maximum columns shown
pd.options.display.max_columns = 100
app.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,306215.000000,306215.000000,306215.000000,3.062150e+05,3.062150e+05,306203.000000,3.062150e+05,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,3.055560e+05,245464.000000,156794.000000,153795.000000,156794.000000,153795.000000,156794.000000,153795.000000,158364.000000,305194.000000,305194.000000,305194.000000,305194.000000,306214.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,306215.000000,264805.000000,264805.000000,264805.000000,264805.000000,264805.000000,264805.000000
mean,278164.519246,0.080842,0.417004,1.687830e+05,5.987997e+05,27122.210470,5.379479e+05,0.020865,-16040.633855,63858.968166,-4987.987728,-2994.331035,0.999997,0.819767,0.199050,0.998126,0.280764,0.056797,2.152778,2.052617,2.031638,2.527104,12.061999,0.015163,0.050749,0.040619,0.078164,0.230492,0.179599,5.143519e-01,0.510923,0.977728,0.226261,0.977056,0.222292,0.977746,0.225877,0.102525,1.421532,0.143374,1.404605,0.100005,-964.425634,0.000042,0.710560,0.000082,0.014715,0.087857,0.000140,0.081342,0.003854,0.000020,0.003840,0.000007,0.003406,0.002805,0.001130,0.009405,0.000261,0.007818,0.000571,0.000493,0.000330,0.006390,0.006982,0.034448,0.267616,0.265697,1.903903
std,102786.814894,0.272593,0.722104,2.375179e+05,4.019606e+05,14490.897429,3.689186e+05,0.013830,4362.856052,141313.558266,3522.557759,1509.518082,0.001807,0.384382,0.399286,0.043255,0.449373,0.231454,0.910584,0.509103,0.502794,1.791450,3.266155,0.122199,0.219484,0.197405,0.268430,0.421148,0.383854,1.910904e-01,0.194836,0.059251,0.144579,0.064624,0.143649,0.059927,0.145009,0.107424,2.400847,0.446637,2.379730,0.362213,826.707866,0.006516,0.453503,0.009035,0.120410,0.283087,0.011849,0.273359,0.061957,0.004426,0.061852,0.002556,0.058262,0.052890,0.033595,0.096523,0.016161,0.088073,0.023899,0.022201,0.018158,0.083791,0.110479,0.204792,0.915624,0.794827,1.869584
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,-25229.000000,-17912.000000,-24672.000000,-7197.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.173617e-08,0.000527,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [30]:
# #DAYS_BIRTH, DAYS_REGISTRATION and DAYS_ID_PUBLISH only have negative values
# app["DAYS_REGISTRATION"] = abs(app["DAYS_REGISTRATION"])
# app["DAYS_ID_PUBLISH"] = abs(app["DAYS_ID_PUBLISH"])
# app["DAYS_BIRTH"] = abs(app["DAYS_BIRTH"])

# #DAYS EMPLOYED have abherrent values (365243 days, about 1000 years)
# app.loc[app["DAYS_EMPLOYED"] > 100000, "DAYS_EMPLOYED"] = np.nan
# app["DAYS_EMPLOYED"] = abs(app["DAYS_EMPLOYED"])

# print(app["DAYS_BIRTH"].min()/365, app["DAYS_BIRTH"].max()/365)
# #No outlier data, from 20 to 69 years

# def label_age(days_birth):
#     age_years = days_birth / 365
#     if age_years < 30: return 1
#     elif age_years < 40: return 2
#     elif age_years < 50: return 3
#     elif age_years < 60: return 4
#     elif age_years < 70: return 5
#     else: return 0
    
# app["AGE_LABEL"] = app["DAYS_BIRTH"].apply(lambda x: label_age(x))

# app = app[app['AMT_INCOME_TOTAL'] < 20000000] # remove an outlier (117 million)

# # Calculated features
# app['DAYS_EMPLOYED_PCT'] = app['DAYS_EMPLOYED'] / app['DAYS_BIRTH']
# app['INCOME_CREDIT_PCT'] = app['AMT_INCOME_TOTAL'] / app['AMT_CREDIT']
# app['INCOME_PER_PERSON'] = app['AMT_INCOME_TOTAL'] / app['CNT_FAM_MEMBERS']
# app['ANNUITY_INCOME_PCT'] = app['AMT_ANNUITY'] / app['AMT_INCOME_TOTAL']
# app['PAYMENT_RATE'] = app['AMT_ANNUITY'] / app['AMT_CREDIT']

20.517808219178082 69.12054794520547


In [31]:
# app.describe()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_LABEL,DAYS_EMPLOYED_PCT,INCOME_CREDIT_PCT,INCOME_PER_PERSON,ANNUITY_INCOME_PCT,PAYMENT_RATE
count,306214.000000,306214.000000,306214.000000,3.062140e+05,3.062140e+05,306202.000000,3.062140e+05,306214.000000,306214.000000,251036.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,3.055550e+05,245463.000000,156793.000000,153794.000000,156793.000000,153794.000000,156793.000000,153794.000000,158363.000000,305193.000000,305193.000000,305193.000000,305193.000000,306213.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,306214.000000,264804.000000,264804.000000,264804.000000,264804.000000,264804.000000,264804.000000,306214.000000,251036.000000,306214.000000,3.062140e+05,306202.000000,306202.000000
mean,278165.052199,0.080839,0.417002,1.684014e+05,5.987998e+05,27122.213500,5.379482e+05,0.020865,16040.645042,2385.328778,4987.981934,2994.328917,0.999997,0.819767,0.199050,0.998125,0.280764,0.056797,2.152776,2.052617,2.031638,2.527108,12.061993,0.015163,0.050749,0.040619,0.078164,0.230492,0.179600,5.143533e-01,0.510925,0.977728,0.226262,0.977056,0.222292,0.977746,0.225878,0.102525,1.421537,0.143375,1.404610,0.100006,-964.428783,0.000042,0.710559,0.000082,0.014715,0.087857,0.000140,0.081342,0.003854,0.000020,0.003840,0.000007,0.003406,0.002805,0.001130,0.009405,0.000261,0.007818,0.000571,0.000493,0.000330,0.006390,0.006983,0.034448,0.267617,0.265698,1.903906,2.893617,0.156905,0.399176,9.296547e+04,0.181055,0.053740
std,102786.559635,0.272588,0.722104,1.088094e+05,4.019612e+05,14490.920994,3.689192e+05,0.013830,4362.858784,2338.980862,3522.562052,1509.520091,0.001807,0.384382,0.399287,0.043255,0.449373,0.231454,0.910585,0.509104,0.502795,1.791451,3.266158,0.122200,0.219485,0.197405,0.268430,0.421149,0.383854,1.910893e-01,0.194835,0.059251,0.144579,0.064624,0.143649,0.059928,0.145009,0.107425,2.400850,0.446638,2.379733,0.362213,826.707378,0.006516,0.453504,0.009035,0.120411,0.283087,0.011849,0.273360,0.061957,0.004426,0.061852,0.002556,0.058263,0.052890,0.033595,0.096523,0.016161,0.088074,0.023899,0.022201,0.018158,0.083792,0.110479,0.204792,0.915626,0.794829,1.869587,1.234927,0.133577,0.343636,7.317446e+04,0.094634,0.022490
min,100002.000000,0.000000,0.000000,2.565000e+04,4.500000e+04,1615.500000,4.050000e+04,0.000290,7

In [32]:
# #Turning SK_ID_CURR into an ID field :
# app.set_index('SK_ID_CURR', inplace=True)

# app.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_LABEL,DAYS_EMPLOYED_PCT,INCOME_CREDIT_PCT,INCOME_PER_PERSON,ANNUITY_INCOME_PCT,PAYMENT_RATE
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,9461,637.0,3648.0,2120,1,1,0,1,1,0,Laborers,1.0,2,2,2,10,0,0,0,0,0,0,Business Entity Type 3,0.262949,0.139376,0.9722,0.0833,0.9722,0.0833,0.9722,0.0833,0.0149,No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,1,0.067329,0.498036,202500.0,0.121978,0.060749
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,16765,1188.0,1186.0,291,1,1,0,1,1,0,Core staff,2.0,1,1,0,11,0,0,0,0,0,0,School,0.622246,NaN,0.9851,0.2917,0.9851,0.2917,0.9851,0.2917,0.0714,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.070862,0.208736,135000.0,0.132217,0.027598
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,19046,225.0,4260.0,2531,1,1,1,1,1,0,Laborers,1.0,2,2,0,9,0,0,0,0,0,0,Government,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.011814,0.500000,67500.0,0.100000,0.050000
100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,19005,3039.0,9833.0,2437,1,1,0,1,0,0,Laborers,2.0,2,2,2,17,0,0,0,0,0,0,Business Entity Type 3,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,4,0.159905,0.431748,67500.0,0.219900,0.094941
100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,19932,3038.0,4311.0,3458,1,1,0,1,0,0,Core staff,1.0,2,2,3,11,0,0,0,0,1,1,Religion,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UKN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,4,0.152418,0.236842,121500.0,0.179963,0.042623


Analysis of the describe() output shows that there is **no clear outlier** in the rest of the numeric data. We can now start handling missing values.

In [33]:
# len(app.columns[app.isnull().any()])
# #21 columns with NA values

# #Dropping rows with more than 50% na values
# def drop_na_rows(df: pd.DataFrame, pct: float):
#     n = len(df.columns)
#     cutoff = n*pct/100 
#     df = df[df.isna().sum(axis=1) > cutoff]

# drop_na_rows(app, 50)
#No row was removed


In [ ]:
#Summarizing cleaning of numeric variables:

def preprocess_num_vars(df: pd.DataFrame, app_train: pd.DataFrame, na_rows_filtered_pct: float):
    #Setting index
    app.set_index('SK_ID_CURR', inplace=True)
    
    #DAYS_BIRTH, DAYS_REGISTRATION and DAYS_ID_PUBLISH only have negative values
    df["DAYS_REGISTRATION"] = abs(df["DAYS_REGISTRATION"])
    df["DAYS_ID_PUBLISH"] = abs(df["DAYS_ID_PUBLISH"])
    df["DAYS_BIRTH"] = abs(df["DAYS_BIRTH"])

    #DAYS EMPLOYED have abherrent values (365243 days, about 1000 years)
    df["DAYS_EMPLOYED_ANOM_FLAG"] = 0 #Creating anomaly flag
    df.loc[df["DAYS_EMPLOYED"] > 100000, "DAYS_EMPLOYED_ANOM_FLAG"] = 1 
    df.loc[df["DAYS_EMPLOYED"] > 100000, "DAYS_EMPLOYED"] = np.nan
    df["DAYS_EMPLOYED"] = abs(df["DAYS_EMPLOYED"])

    print(df["DAYS_BIRTH"].min()/365, df["DAYS_BIRTH"].max()/365)
    #No outlier data, from 20 to 69 years

    def label_age(days_birth):
        age_years = days_birth / 365
        if age_years < 30: return 1
        elif age_years < 40: return 2
        elif age_years < 50: return 3
        elif age_years < 60: return 4
        elif age_years < 70: return 5
        else: return 0

    df["AGE_LABEL"] = df["DAYS_BIRTH"].apply(lambda x: label_age(x))

    df = df[df['AMT_INCOME_TOTAL'] < 20000000] # remove an outlier (117 million)
    
    #Selecting quantiles from app_train for uniformisation
    income_quantiles = [app_train["AMT_INCOME_TOTAL"].quantile(i/10) for i in range(0,11,1)]
    
    df["AMT_INCOME_BIN"] = pd.cut(df["AMT_INCOME_TOTAL"], bins=income_quantiles, labels=False)

    # Calculated features
    df['DAYS_EMPLOYED_PCT'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PCT'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PCT'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    df['PAYMENT_RATE'] = df['AMT_ANNUITY'] / df['AMT_CREDIT']
    

We've now finished cleaning incorrect values. 
Before starting to perform data imputation, we need to perform a **train/validation/test split**. This will **prevent us from introducing data leakage during the cleaning process**. 

### 2.3 Performing train / test / validation split

We will divide our dataset as such : 

-  80% train set 
-  10% validation 
-  10% test

We will be able to revisit this values during the hyperparameter tuning part of the project.

In [34]:
from sklearn.model_selection import train_test_split

def perform_split(df: pd.DataFrame, target_col: str, validation_set: bool, shuffle: bool, random_state: int, test_size: float):

    y = df["TARGET"]
    ID = df.index
    X = df.drop(columns={"TARGET"})

    #Splitting train and test sets, we have to add indices to conserve the original index
    X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(
        X, y, ID, test_size=test_size, stratify=y, shuffle=shuffle, random_state=random_state)

    #Assigning the correct indices (the SK_IDs) to y_test
    y_test.index = indices_test
    
    if validation_set:
        #Applying the same function to separate train and validation set
        X_train, X_val, y_train, y_val, indices_train, indices_val = train_test_split(
            X_train, y_train, indices_train, test_size = test_size/(1-test_size), stratify=y, 
            shuffle=shuffle, random_state=random_state)

        #Assigning the SK IDs to y_train and y_val
        y_val.index = indices_val
    
    y_train.index = indices_train


In [35]:
# print(len(X_train), len(X_test), len(X_val))
#Our test and validation set have the same length and its 10% of the overall length of X

244970 30622 30622


Now that we have performed the split, we can carry on to perform data imputation.

These operations will also have to be performed on the test and train_set, so we will create a function that we will be able to apply to the 3 sets.

### 2.4 Data Imputation

First we will investigate what columns still have missing values. 
Normally, we have replaced all missing features for categorical variables.

In [36]:
# #For ease of use, we will rename X_train to df so we can better replicate our code afterwards
# df = X_train.copy()

# def check_col_nas_type(df: pd.DataFrame):
#     type_cols = []
#     #Verifying the type of columns with missing values
#     for c in df.columns[df.isna().any()].tolist():
#         if ~np.isin(df[c].dtype, type_cols):
#             type_cols.append(df[c].dtype)
#     return(type_cols)

# check_col_nas_type(df)
# #This verifies that we only need to perform data imputation on numeric features

In [37]:
# #Loading visualization functions present in the functions.py file
# from functions import *

# #Visualizing distribution of all numeric variables
# histPlotAll(df)

# #Apart from HOUR_APPR_PROCESS_START, all numeric variables seem to be not normally distributed

In [38]:
from scipy import stats

#Defining a data imputation function, we will use the NAME_CONTRACT_TYPE as a category_column

#This data_imputation script can be improved during the hyperparameter setting phase

def numeric_data_imputation(df: pd.DataFrame, max_unique_values=3, use_categ_column=False, category_column=None):
    
    #Creating a copy of our dataset
    df_imput = df.copy()
    #Creating a list of columns with missing values
    missing_cols = df.columns[df.isna().any()].tolist()
    
    #Iterating over columns with missing data
    for c in missing_cols:
        
        #Verifying that we are in a numeric column
        if np.issubdtype(df[c].dtype,np.number):
            
            #If there are less or equal to max unique values, we will use mode imputation 
            if len(df[c].unique()) <= max_unique_values:
                
                if use_categ_column:
                            
                    #We will create a subset from our categorical variable and perform mode imputation
                    for t in df[category_column].unique():
                        #Creating subset
                        subset = df.loc[df[category_column] == t]

                        #Calculating mode of subset
                        mode = subset[c].mode()

                        #Applying imputation
                        df.loc[(df[c].isna()) & (df[category_column] == t), c] = mode
                
                else:
                    df[c] = df[c].fillna(df[c].mode())
                            
            #If we have more numeric values, we will calculate the Kolmogorov Smirnoff pvalue to test for normalization
            else:
                
                #Normalizing target variable
                norm = c + '_norm'
                df_imput[norm] = (df_imput[c] - np.mean(df_imput[c].dropna())) / np.std(df_imput[c].dropna())

                #Calculating pvalue of KS test
                pval = stats.kstest(df_imput[norm].dropna(), 'norm').pvalue
                
                if pval >= 0.05:
                    
                        if use_categ_column:
                        #P value is superior to 0.05, we cannot reject the null hypothesis and thus conclude the variable is
                        #approximatively normally distributed
                        #We will use mean imputation on that variable
                        for t in df[category_column].unique():
                            #Creating subset
                            subset = df.loc[df[category_column] == t]

                            #Calculating mean based on that subset and our target column
                            mean = subset[c].mean()

                            #Applying imputation
                            df.loc[(df[c].isna()) & (df[category_column] == t), c] = mean
                            
                        else:
                            df[c] = df[c].fillna(df[c].mean())
                            
                else:
                    
                    if use_categ_column:
                        
                        #P value is inferior to 0.05, we can reject the null hypothesis and thus conclude the variable is
                        #not normally distributed
                        #We will use median imputation on that variable
                        for t in df[category_column].unique():
                            #Creating subset
                            subset = df.loc[df[category_column] == t]

                            #Calculating mean based on that subset and our target column
                            med = subset[c].median()

                            #Applying imputation
                            df.loc[(df[c].isna()) & (df[category_column] == t), c] = med
                            
                    else:
                        df[c] = df[c].fillna(df[c].median())
    return None

# #Applying the function to our 3 sets (X_train has been renamed to df)
# numeric_data_imputation(df, 'NAME_CONTRACT_TYPE')
# numeric_data_imputation(X_test, 'NAME_CONTRACT_TYPE')
# numeric_data_imputation(X_val, 'NAME_CONTRACT_TYPE')

#Checking for nulls in our 3 sets
# for data in [df,X_test,X_val]:
#     print(np.count_nonzero(data.isnull()))
    
#We have no more NA values in all 3 sets

Now that we have 3 complete datasets, we can perform **feature engineering**

## 3. Feature Engineering

We will begin by encoding cyclical features.

### 3.1 Encoding Cyclical Features

We have 2 columns with time features that are cyclical in nature but coded with numbers.

- WEEDKAY_APPR_PROCESS_START
- HOUR_APPR_PROCESS_START

To increase the performance of our algorithm, we will apply a cyclical encoding algorithm to better represent their cyclical nature :

In [39]:
def encode_cyclical_vars(df: pd.DataFrame, cyclical_vars=["WEEKDAY_APPR_PROCESS_START", "HOUR_APPR_PROCESS_START"]):
    for c in cyclical_vars:
        #Calculating the number of unique values
        n = len(df[c].unique())
        #Defining variable names
        cos_var = c + '_cos'
        sin_var = c + '_sin'
        #Calculating cyclical encoder variables
        df[sin_var] = np.sin(df[c] * (2*np.pi/n))
        df[cos_var] = np.cos(df[c] * (2*np.pi/n))
        #Dropping the base columns
        df.drop(columns = {c}, inplace=True)

# encode_cyclical_vars(df)
# encode_cyclical_vars(X_test)
# encode_cyclical_vars(X_val)

# df.head()

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_2,EXT_SOURCE_3,YEARS_BEGINEXPLUATATION_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,FLOORSMAX_MODE,YEARS_BEGINEXPLUATATION_MEDI,FLOORSMAX_MEDI,TOTALAREA_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,AGE_LABEL,DAYS_EMPLOYED_PCT,INCOME_CREDIT_PCT,INCOME_PER_PERSON,ANNUITY_INCOME_PCT,PAYMENT_RATE,WEEKDAY_APPR_PROCESS_START_sin,WEEKDAY_APPR_PROCESS_START_cos,HOUR_APPR_PROCESS_START_sin,HOUR_APPR_PROCESS_START_cos
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
369727,Cash loans,F,N,Y,0,292500.0,970380.0,25726.5,810000.0,Unaccompanied,Commercial associate,Higher education,Civil marriage,House / apartment,0.024610,20291,951.0,1544.0,1054,1,1,1,1,1,0,Laborers,2.0,2,2,0,0,0,0,0,0,Transport: type 4,0.653810,0.103449,0.9925,0.1667,0.9926,0.1667,0.9925,0.1667,0.0715,No,0.0,0.0,0.0,0.0,-1026.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,4,0.046868,0.301428,146250.0,0.087954,0.026512,-0.974928,-0.222521,2.588190e-01,-0.965926
271823,Cash loans,F,N,Y,0,202500.0,904500.0,36000.0,904500.0,Unaccompanied,Working,Secondary / secondary special,Separated,House / apartment,0.025164,16735,3442.0,6006.0,267,1,1,1,1,0,0,Laborers,1.0,2,2,0,0,0,0,0,0,Industry: type 1,0.159688,0.522697,0.0000,0.0417,0.0005,0.0417,0.0000,0.0417,0.0166,No,1.0,1.0,1.0,1.0,-1752.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0,3,0.205677,0.223881,202500.0,0.177778,0.039801,-0.433884,-0.900969,2.588190e-01,-0.965926
136775,Cash loans,F,N,Y,0,270000.0,1035000.0,30393.0,1035000.0,Unaccompanied,Commercial associate,Incomplete higher,Married,House / apartment,0.024610,16141,188.0,283.0,4636,1,1,0,1,0,0,UKN,2.0,2,2,0,0,0,0,0,0,Agriculture,0.704620,0.586740,0.9816,0.1667,0.9816,0.1667,0.9816,0.1667,0.0685,UKN,0.0,0.0,0.0,0.0,-1036.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,1.0,1.0,3,0.011647,0.260870,135000.0,0.112567,0.029365,-0.433884,-0.900969,-8.660254e-01,-0.500000
329676,Cash loans,M,N,N,0,112500.0,144000.0,9450.0,144000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Single / not married,Rented apartment,0.014520,15108,1096.0,6018.0,4210,1,1,0,1,0,0,Core staff,1.0,2,2,0,0,0,1,1,0,Security,0.344191,0.526295,0.9846,0.0554,0.9816,0.0417,0.9816,0.0417,0.0126,No,0.0,0.0,0.0,0.0,-737.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,2.0,1.0,3,0.072544,0.781250,112500.0,0.084000,0.065625,0.000000,1.000000,7.071068e-01,-0.707107
192999,Cash loans,F,Y,Y,0,198000.0,704844.0,34038.0,630000.0,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,0.011703,19811,1714.0,140.0,3339,1,1,0,1,0,0,Laborers,2.0,2,2,0,0,0,0,1,1,Other,0.617299,0.735221,0.9816,0.1667,0.9816,0.1667,0.9816,0.1667,0.0685,UKN,4.0,0.0,4.

### 3.2 Encoding categorical variables

Since our algorithms are only able to use numeric variables, we will need to **encode categorical variables**.

For variables with a small number of categories, we will perform **One-Hot Encoding**.

If there are more than 10 categories, we will perform **Weight of Evidence (WoE) encoding** instead to avoid a sharp increase in the dimensionality of our dataset.

In [40]:
from category_encoders import WOEEncoder
from category_encoders.one_hot import OneHotEncoder
from category_encoders.binary import BinaryEncoder
from category_encoders.ordinal import OrdinalEncoder


def encode_cat_vars(df: pd.DataFrame, X_train: pd.DataFrame, y_train, max_categ: int, drop_invariant=False, woe_encode=True):
    woe_cols = []
    ohe_cols = []
    label_cols = []
    for c in X_train.columns:
        
        #Keeping only categorical columns
        if not np.issubdtype(X_train[c].dtype,np.number):
            
            #If only 2 categories, performing Label encoding
            if len(X_train[c].unique()) == 2:
                label_cols.append(c)
            
            #If more than X categories, performing WOE encoding
            elif len(X_train[c].unique()) >= max_categ:
                woe_cols.append(c)
            
            else: 
                #One hot encoding and remove the original column
                ohe_cols.append(c)
                
    #Defining Binary Encoder based on the train dataset and applying it to df
    bin_enc = BinaryEncoder(cols= label_cols, drop_invariant=drop_invariant, return_df=True).fit(X_train)
    X_train_encoded = bin_enc.transform(X_train)
    df = bin_enc.transform(df)
    
    if woe_encode:
        #Defining WOE Encoder and fitting it to the TRAIN dataset
        woe_encoder = WOEEncoder(cols = woe_cols, drop_invariant=drop_invariant, return_df=True).fit(X_train_encoded, y_train)
        X_train_encoded = woe_encoder.transform(X_train_encoded)
        #Fitting the encoder to the selected dataframe
        df = woe_encoder.transform(df)
    else: #Perform label (ordinal) encoding
        label_encoder = LabelEncoder(cols=woe_cols, drop_invariant=drop_invariant, return_df=True).fit(X_train_encoded)
        X_train encoded = label_encoder.transform(X_train_encoded)
        df = label_encoder.transform(df)
    
    #Performing one hot encoding on selected columns
    ohe_encoder = OneHotEncoder(cols=ohe_cols, return_df= True, drop_invariant=drop_invariant).fit(X_train_encoded)
    df = ohe_encoder.transform(df)
    
    
    del X_train_encoded
    return df

#Just a reminder that once again df = X_train
#We apply all this function to our 3 sets
# X_test = encode_cat_vars(X_test, df, y_train, 10)
# X_val = encode_cat_vars(X_val, df, y_train, 10)
# df = encode_cat_vars(df, df, y_train, 10)

In [41]:
# def check_dtypes(df: pd.DataFrame):
#     type_cols = []
#     for c in df.columns:
#         if not np.isin(df[c].dtype, type_cols):
#             type_cols.append(df[c].dtype)
#     print(type_cols)

# check_dtypes(df)
# check_dtypes(X_test)
# check_dtypes(X_val)

[dtype('int64'), dtype('float64')]
[dtype('int64'), dtype('float64')]
[dtype('int64'), dtype('float64')]


In [42]:
# print(df.shape, X_test.shape, X_val.shape)

(244970, 119) (30622, 119) (30622, 119)


We have verified that all of our 3 sets are composed only of numeric features and that they have the same number of columns.

We will now use **additional features from other dataframes** to increase the performance of our models.

### 3.3 Using previous application data

In [78]:
# prev_app = pd.read_csv("Data/previous_application.csv", sep=",")

# prev_app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1670214 entries, 0 to 1670213
Data columns (total 37 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   SK_ID_PREV                   1670214 non-null  int64  
 1   SK_ID_CURR                   1670214 non-null  int64  
 2   NAME_CONTRACT_TYPE           1670214 non-null  object 
 3   AMT_ANNUITY                  1297979 non-null  float64
 4   AMT_APPLICATION              1670214 non-null  float64
 5   AMT_CREDIT                   1670213 non-null  float64
 6   AMT_DOWN_PAYMENT             774370 non-null   float64
 7   AMT_GOODS_PRICE              1284699 non-null  float64
 8   WEEKDAY_APPR_PROCESS_START   1670214 non-null  object 
 9   HOUR_APPR_PROCESS_START      1670214 non-null  int64  
 10  FLAG_LAST_APPL_PER_CONTRACT  1670214 non-null  object 
 11  NFLAG_LAST_APPL_IN_DAY       1670214 non-null  int64  
 12  RATE_DOWN_PAYMENT            774370 non-nu

In [9]:
# prev_app.describe()

In [79]:
# #We are interested in DAYS_LAST_DUE (the number of days the borrower has to pay previous applications)
# #But there are illogical values (365243 which is equal to 1000 years)
# #First we'll replace all the values by nan
# prev_app.loc[prev_app.DAYS_LAST_DUE > 300000, "DAYS_LAST_DUE"] = np.nan
# prev_app.loc[prev_app.DAYS_FIRST_DUE > 300000, "DAYS_FIRST_DUE"] = np.nan
# prev_app.loc[prev_app.DAYS_LAST_DUE_1ST_VERSION > 300000, "DAYS_LAST_DUE_1ST_VERSION"] = np.nan
# prev_app.loc[prev_app.DAYS_FIRST_DRAWING > 300000, "DAYS_FIRST_DRAWING"] = np.nan
# prev_app.loc[prev_app.DAYS_TERMINATION > 300000, "DAYS_TERMINATION"] = np.nan

# #Defining current amount due, we have to add a negative sign because DAYS_LAST_DUE is negative
# prev_app["AMT_CURR_DUE"] = -prev_app["AMT_ANNUITY"]*prev_app["DAYS_LAST_DUE"]/365

# prev_app["CURR_ANNUITY"] = 0
# prev_app.loc[prev_app["DAYS_LAST_DUE"] < 0, "CURR_ANNUITY"] = prev_app["AMT_ANNUITY"]

# # Calculated variables
# prev_app['APPLICATION_CREDIT_DIF'] = prev_app['AMT_APPLICATION'] - prev_app['AMT_CREDIT']
# prev_app['CREDIT_TO_ANNUITY'] = prev_app['AMT_CREDIT'] / prev_app['AMT_ANNUITY']
# prev_app['DOWN_PAYMENT_TO_CREDIT'] = prev_app['AMT_DOWN_PAYMENT'] / prev_app['AMT_CREDIT']

# prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,AMT_CURR_DUE,CURR_ANNUITY,APPLICATION_CREDIT_DIF,CREDIT_TO_ANNUITY,DOWN_PAYMENT_TO_CREDIT
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,NaN,-42.0,300.0,-42.0,-37.0,0.0,199.117973,1730.430,0.0,9.907942,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,NaN,-134.0,916.0,NaN,NaN,1.0,NaN,0.000,-72171.0,26.983262,NaN
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,NaN,-271.0,59.0,NaN,NaN,1.0,NaN,0.000,-23944.5,9.059618,NaN
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,NaN,-482.0,-152.0,-182.0,-177.0,1.0,23456.227315,47041.335,-20790.0,10.008007,NaN
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,-66555.0,12.656622,NaN


In [80]:
# #Verifying unique values of contract status
# prev_app.NAME_CONTRACT_STATUS.unique()
# #4 categories, Approved, Refused, Canceled and Unused offer

# prev_app["AMT_GRANTED"] = 0
# prev_app.loc[prev_app["NAME_CONTRACT_STATUS"] == "Approved", "AMT_GRANTED"] = prev_app["AMT_CREDIT"]

# prev_app.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,FLAG_LAST_APPL_PER_CONTRACT,NFLAG_LAST_APPL_IN_DAY,RATE_DOWN_PAYMENT,RATE_INTEREST_PRIMARY,RATE_INTEREST_PRIVILEGED,NAME_CASH_LOAN_PURPOSE,NAME_CONTRACT_STATUS,DAYS_DECISION,NAME_PAYMENT_TYPE,CODE_REJECT_REASON,NAME_TYPE_SUITE,NAME_CLIENT_TYPE,NAME_GOODS_CATEGORY,NAME_PORTFOLIO,NAME_PRODUCT_TYPE,CHANNEL_TYPE,SELLERPLACE_AREA,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL,AMT_CURR_DUE,CURR_ANNUITY,APPLICATION_CREDIT_DIF,CREDIT_TO_ANNUITY,DOWN_PAYMENT_TO_CREDIT,AMT_GRANTED
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,Y,1,0.0,0.182832,0.867336,XAP,Approved,-73,Cash through the bank,XAP,NaN,Repeater,Mobile,POS,XNA,Country-wide,35,Connectivity,12.0,middle,POS mobile with interest,NaN,-42.0,300.0,-42.0,-37.0,0.0,199.117973,1730.430,0.0,9.907942,0.0,17145.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-164,XNA,XAP,Unaccompanied,Repeater,XNA,Cash,x-sell,Contact center,-1,XNA,36.0,low_action,Cash X-Sell: low,NaN,-134.0,916.0,NaN,NaN,1.0,NaN,0.000,-72171.0,26.983262,NaN,679671.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,Y,1,NaN,NaN,NaN,XNA,Approved,-301,Cash through the bank,XAP,"Spouse, partner",Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,high,Cash X-Sell: high,NaN,-271.0,59.0,NaN,NaN,1.0,NaN,0.000,-23944.5,9.059618,NaN,136444.5
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,Y,1,NaN,NaN,NaN,XNA,Approved,-512,Cash through the bank,XAP,NaN,Repeater,XNA,Cash,x-sell,Credit and cash offices,-1,XNA,12.0,middle,Cash X-Sell: middle,NaN,-482.0,-152.0,-182.0,-177.0,1.0,23456.227315,47041.335,-20790.0,10.008007,NaN,470790.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,Y,1,NaN,NaN,NaN,Repairs,Refused,-781,Cash through the bank,HC,NaN,Repeater,XNA,Cash,walk-in,Credit and cash offices,-1,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,-66555.0,12.656622,NaN,0.0


In [81]:
# aggregations = {
#         'AMT_ANNUITY': ['std', 'mean', 'sum'],
#         'AMT_APPLICATION': ['std', 'mean', 'sum'],
#         'AMT_CREDIT': ['std', 'mean', 'sum'],
#         'AMT_CURR_DUE': ['std', 'mean', 'sum'],
#         'CURR_ANNUITY': ['std', 'mean', 'sum'],
#         'AMT_DOWN_PAYMENT': ['std', 'mean', 'sum'],
#         'AMT_GOODS_PRICE': ['std', 'mean', 'sum'],
#         'HOUR_APPR_PROCESS_START': ['std', 'mean'],
#         'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
#         'DAYS_DECISION': ['std', 'mean', 'sum'],
#         'CNT_PAYMENT': ['mean', 'sum','std'],
#         'SK_ID_PREV': ['nunique'],
#         'DAYS_TERMINATION': ['mean', 'sum', 'std'],
#         'DOWN_PAYMENT_TO_CREDIT': ['sum', 'mean', 'std']
#     }

# #We will aggregate by SK_ID_CURR and retrieve important information about previous applications :
# prev_app_numbers = prev_app.groupby("SK_ID_CURR").agg(aggregations).fillna(0)

# #Renaming columns to remove multi indexing
# prev_app_numbers.columns = pd.Index(['APP' + '_' + e[0] + '_' + e[1] for e in prev_app_numbers.columns])

# prev_app_numbers.head()

,APP_AMT_ANNUITY_std,APP_AMT_ANNUITY_mean,APP_AMT_ANNUITY_sum,APP_AMT_APPLICATION_std,APP_AMT_APPLICATION_mean,APP_AMT_APPLICATION_sum,APP_AMT_CREDIT_std,APP_AMT_CREDIT_mean,APP_AMT_CREDIT_sum,APP_AMT_CURR_DUE_std,APP_AMT_CURR_DUE_mean,APP_AMT_CURR_DUE_sum,APP_CURR_ANNUITY_std,APP_CURR_ANNUITY_mean,APP_CURR_ANNUITY_sum,APP_AMT_DOWN_PAYMENT_std,APP_AMT_DOWN_PAYMENT_mean,APP_AMT_DOWN_PAYMENT_sum,APP_AMT_GOODS_PRICE_std,APP_AMT_GOODS_PRICE_mean,APP_AMT_GOODS_PRICE_sum,APP_HOUR_APPR_PROCESS_START_std,APP_HOUR_APPR_PROCESS_START_mean,APP_RATE_DOWN_PAYMENT_min,APP_RATE_DOWN_PAYMENT_max,APP_RATE_DOWN_PAYMENT_mean,APP_DAYS_DECISION_std,APP_DAYS_DECISION_mean,APP_DAYS_DECISION_sum,APP_CNT_PAYMENT_mean,APP_CNT_PAYMENT_sum,APP_CNT_PAYMENT_std,APP_SK_ID_PREV_nunique,APP_DAYS_TERMINATION_mean,APP_DAYS_TERMINATION_sum,APP_DAYS_TERMINATION_std,APP_DOWN_PAYMENT_TO_CREDIT_sum,APP_DOWN_PAYMENT_TO_CREDIT_mean,APP_DOWN_PAYMENT_TO_CREDIT_std
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.000000,3951.000,3951.000,0.000000,24835.50,24835.5,0.000000,23787.00,23787.0,0.000000,17525.120548,17525.120548,0.000000,3951.000,3951.000,0.000000,2520.0,2520.0,0.000000,24835.5,24835.5,0.000000,13.000000,0.104326,0.104326,0.104326,0.000000,-1740.0,-1740,8.0,8.0,0.000000,1,-1612.000000,-1612.0,0.000000,0.105940,0.105940,0.000000
100002,0.000000,9251.775,9251.775,0.000000,179055.00,179055.0,0.000000,179055.00,179055.0,0.000000,633.683219,633.683219,0.000000,9251.775,9251.775,0.000000,0.0,0.0,0.000000,179055.0,179055.0,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,-606.0,-606,24.0,24.0,0.000000,1,-17.000000,-17.0,0.000000,0.000000,0.000000,0.000000
100003,46332.557777,56553.990,169661.970,424161.620549,435436.50,1306309.5,497949.861808,484191.00,1452573.0,55690.034366,98478.997603,295436.992808,46332.557777,56553.990,169661.970,4868.430188,3442.5,6885.0,424161.620549,435436.5,1306309.5,2.516611,14.666667,0.000000,0.100061,0.050030,898.138631,-1305.0,-3915,10.0,30.0,3.464102,3,-1047.333333,-3142.0,806.196213,0.101170,0.050585,0.071538
100004,0.000000,5357.250,5357.250,0.000000,24282.00,24282.0,0.000000,20106.00,20106.0,0.000000,10626.435616,10626.435616,0.000000,5357.250,5357.250,0.000000,4860.0,4860.0,0.000000,24282.0,24282.0,0.000000,5.000000,0.212008,0.212008,0.212008,0.000000,-815.0,-815,4.0,4.0,0.000000,1,-714.000000,-714.0,0.000000,0.241719,0.241719,0.000000
100005,0.000000,4813.200,4813.200,31549.336810,22308.75,44617.5,28392.812138,20076.75,40153.5,0.000000,6145.071781,6145.071781,3403.446359,2406.600,4813.200,0.000000,4464.0,4464.0,0.000000,44617.5,44617.5,0.707107,10.500000,0.108964,0.108964,0.108964,312.541197,-536.0,-1072,12.0,12.0,0.000000,2,-460.000000,-460.0,0.000000,0.111173,0.111173,0.000000


In [82]:
# #Creating a dataframe with the number of each different name contract status by SK_ID_CURR
# prev_app_status = pd.crosstab(prev_app['SK_ID_CURR'], prev_app['NAME_CONTRACT_STATUS'])

# cols = ["N_PREV_APPROVED","N_PREV_CANCELED","N_PREV_REFUSED","N_PREV_UNUSED"]
# prev_app_status.columns = cols

# #Importing the number of unique applications from prev_app_numbers
# prev_app_status = pd.merge(prev_app_status, prev_app_numbers[["APP_SK_ID_PREV_nunique"]],
#                            how="inner", left_index=True, right_index=True)

# for c in cols:
#     prev_app_status[c] = prev_app_status[c] / prev_app_status["APP_SK_ID_PREV_nunique"]

# prev_app_status = prev_app_status.drop(columns={"APP_SK_ID_PREV_nunique"})
# prev_app_status.head()

,n_prev_app_approved,n_prev_app_canceled,n_prev_app_refused,n_prev_app_unused
SK_ID_CURR,,,,
100001,1.0,0.0,0.0,0.0
100002,1.0,0.0,0.0,0.0
100003,1.0,0.0,0.0,0.0
100004,1.0,0.0,0.0,0.0
100005,0.5,0.5,0.0,0.0


In [83]:
# prev_app_df = pd.merge(prev_app_numbers, prev_app_status, how='inner', left_index=True, right_index=True)

# prev_app_df.head()

,APP_AMT_ANNUITY_std,APP_AMT_ANNUITY_mean,APP_AMT_ANNUITY_sum,APP_AMT_APPLICATION_std,APP_AMT_APPLICATION_mean,APP_AMT_APPLICATION_sum,APP_AMT_CREDIT_std,APP_AMT_CREDIT_mean,APP_AMT_CREDIT_sum,APP_AMT_CURR_DUE_std,APP_AMT_CURR_DUE_mean,APP_AMT_CURR_DUE_sum,APP_CURR_ANNUITY_std,APP_CURR_ANNUITY_mean,APP_CURR_ANNUITY_sum,APP_AMT_DOWN_PAYMENT_std,APP_AMT_DOWN_PAYMENT_mean,APP_AMT_DOWN_PAYMENT_sum,APP_AMT_GOODS_PRICE_std,APP_AMT_GOODS_PRICE_mean,APP_AMT_GOODS_PRICE_sum,APP_HOUR_APPR_PROCESS_START_std,APP_HOUR_APPR_PROCESS_START_mean,APP_RATE_DOWN_PAYMENT_min,APP_RATE_DOWN_PAYMENT_max,APP_RATE_DOWN_PAYMENT_mean,APP_DAYS_DECISION_std,APP_DAYS_DECISION_mean,APP_DAYS_DECISION_sum,APP_CNT_PAYMENT_mean,APP_CNT_PAYMENT_sum,APP_CNT_PAYMENT_std,APP_SK_ID_PREV_nunique,APP_DAYS_TERMINATION_mean,APP_DAYS_TERMINATION_sum,APP_DAYS_TERMINATION_std,APP_DOWN_PAYMENT_TO_CREDIT_sum,APP_DOWN_PAYMENT_TO_CREDIT_mean,APP_DOWN_PAYMENT_TO_CREDIT_std,n_prev_app_approved,n_prev_app_canceled,n_prev_app_refused,n_prev_app_unused
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.000000,3951.000,3951.000,0.000000,24835.50,24835.5,0.000000,23787.00,23787.0,0.000000,17525.120548,17525.120548,0.000000,3951.000,3951.000,0.000000,2520.0,2520.0,0.000000,24835.5,24835.5,0.000000,13.000000,0.104326,0.104326,0.104326,0.000000,-1740.0,-1740,8.0,8.0,0.000000,1,-1612.000000,-1612.0,0.000000,0.105940,0.105940,0.000000,1.0,0.0,0.0,0.0
100002,0.000000,9251.775,9251.775,0.000000,179055.00,179055.0,0.000000,179055.00,179055.0,0.000000,633.683219,633.683219,0.000000,9251.775,9251.775,0.000000,0.0,0.0,0.000000,179055.0,179055.0,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,-606.0,-606,24.0,24.0,0.000000,1,-17.000000,-17.0,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0
100003,46332.557777,56553.990,169661.970,424161.620549,435436.50,1306309.5,497949.861808,484191.00,1452573.0,55690.034366,98478.997603,295436.992808,46332.557777,56553.990,169661.970,4868.430188,3442.5,6885.0,424161.620549,435436.5,1306309.5,2.516611,14.666667,0.000000,0.100061,0.050030,898.138631,-1305.0,-3915,10.0,30.0,3.464102,3,-1047.333333,-3142.0,806.196213,0.101170,0.050585,0.071538,1.0,0.0,0.0,0.0
100004,0.000000,5357.250,5357.250,0.000000,24282.00,24282.0,0.000000,20106.00,20106.0,0.000000,10626.435616,10626.435616,0.000000,5357.250,5357.250,0.000000,4860.0,4860.0,0.000000,24282.0,24282.0,0.000000,5.000000,0.212008,0.212008,0.212008,0.000000,-815.0,-815,4.0,4.0,0.000000,1,-714.000000,-714.0,0.000000,0.241719,0.241719,0.000000,1.0,0.0,0.0,0.0
100005,0.000000,4813.200,4813.200,31549.336810,22308.75,44617.5,28392.812138,20076.75,40153.5,0.000000,6145.071781,6145.071781,3403.446359,2406.600,4813.200,0.000000,4464.0,4464.0,0.000000,44617.5,44617.5,0.707107,10.500000,0.108964,0.108964,0.108964,312.541197,-536.0,-1072,12.0,12.0,0.000000,2,-460.000000,-460.0,0.000000,0.111173,0.111173,0.000000,0.5,0.5,0.0,0.0


In [84]:
# #Saving prev_app_df to prevent RAM usage and reduce rerun time
# prev_app_df.to_csv("Data/prev_app_df.csv")

In [85]:
def load_prev_app_info(df, X_test, X_val=None, validation_set=False):

    #Loading prev_app_df
    prev_app_df = pd.read_csv("Data/prev_app_df.csv")

    #Joining this new data and filling NAs with 0 (since it means there was no previous application)
    df = pd.merge(df, prev_app_df, how='left', left_index=True, right_index=True).fillna(0)
    X_test = pd.merge(X_test, prev_app_df, how='left', left_index=True, right_index=True).fillna(0)
    if validation_set:
        X_val = pd.merge(X_val, prev_app_df, how='left', left_index=True, right_index=True).fillna(0)


### 3.4 Using Credit Bureau information

We also have information about CB for each borrower that we can use to increase the accuracy of our model:

In [86]:
# bureau = pd.read_csv("Data/bureau.csv", sep=",")

# bureau.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1716428 entries, 0 to 1716427
Data columns (total 17 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   SK_ID_CURR              int64  
 1   SK_ID_BUREAU            int64  
 2   CREDIT_ACTIVE           object 
 3   CREDIT_CURRENCY         object 
 4   DAYS_CREDIT             int64  
 5   CREDIT_DAY_OVERDUE      int64  
 6   DAYS_CREDIT_ENDDATE     float64
 7   DAYS_ENDDATE_FACT       float64
 8   AMT_CREDIT_MAX_OVERDUE  float64
 9   CNT_CREDIT_PROLONG      int64  
 10  AMT_CREDIT_SUM          float64
 11  AMT_CREDIT_SUM_DEBT     float64
 12  AMT_CREDIT_SUM_LIMIT    float64
 13  AMT_CREDIT_SUM_OVERDUE  float64
 14  CREDIT_TYPE             object 
 15  DAYS_CREDIT_UPDATE      int64  
 16  AMT_ANNUITY             float64
dtypes: float64(8), int64(6), object(3)
memory usage: 222.6+ MB


In [87]:
# bureau.describe()

,SK_ID_CURR,SK_ID_BUREAU,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
count,1.716428e+06,1.716428e+06,1.716428e+06,1.716428e+06,1.610875e+06,1.082775e+06,5.919400e+05,1.716428e+06,1.716415e+06,1.458759e+06,1.124648e+06,1.716428e+06,1.716428e+06,4.896370e+05
mean,2.782149e+05,5.924434e+06,-1.142108e+03,8.181666e-01,5.105174e+02,-1.017437e+03,3.825418e+03,6.410406e-03,3.549946e+05,1.370851e+05,6.229515e+03,3.791276e+01,-5.937483e+02,1.571276e+04
std,1.029386e+05,5.322657e+05,7.951649e+02,3.654443e+01,4.994220e+03,7.140106e+02,2.060316e+05,9.622391e-02,1.149811e+06,6.774011e+05,4.503203e+04,5.937650e+03,7.207473e+02,3.258269e+05
min,1.000010e+05,5.000000e+06,-2.922000e+03,0.000000e+00,-4.206000e+04,-4.202300e+04,0.000000e+00,0.000000e+00,0.000000e+00,-4.705600e+06,-5.864061e+05,0.000000e+00,-4.194700e+04,0.000000e+00
25%,1.888668e+05,5.463954e+06,-1.666000e+03,0.000000e+00,-1.138000e+03,-1.489000e+03,0.000000e+00,0.000000e+00,5.130000e+04,0.000000e+00,0.000000e+00,0.000000e+00,-9.080000e+02,0.000000e+00
50%,2.780550e+05,5.926304e+06,-9.870000e+02,0.000000e+00,-3.300000e+02,-8.970000e+02,0.000000e+00,0.000000e+00,1.255185e+05,0.000000e+00,0.000000e+00,0.000000e+00,-3.950000e+02,0.000000e+00
75%,3.674260e+05,6.385681e+06,-4.740000e+02,0.000000e+00,4.740000e+02,-4.250000e+02,0.000000e+00,0.000000e+00,3.150000e+05,4.015350e+04,0.000000e+00,0.000000e+00,-3.300000e+01,1.350000e+04
max,4.562550e+05,6.843457e+06,0.000000e+00,2.792000e+03,3.119900e+04,0.000000e+00,1.159872e+08,9.000000e+00,5.850000e+08,1.701000e+08,4.705600e+06,3.756681e+06,3.720000e+02,1.184534e+08


In [ ]:
#print(bureau.CREDIT_ACTIVE.unique())
# print(bureau.CREDIT_CURRENCY.unique())

# len(bureau[bureau.CREDIT_CURRENCY.isna()])
# #Credit active is interesting because of the bad debt field
# #Currency is also interesting because it could be an indicator to fraudulent transactions

In [93]:
# #Calculating new features

# #Date differences
# bureau['CREDIT_DURATION'] = -bureau['DAYS_CREDIT'] + bureau['DAYS_CREDIT_ENDDATE']
# bureau['ENDDATE_DIF'] = bureau['DAYS_CREDIT_ENDDATE'] - bureau['DAYS_ENDDATE_FACT']

# #Day overdue flags:
# bureau['BUREAU_IS_DPD'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 0 else 0)
# bureau['BUREAU_IS_DPD_OVER100'] = bureau['CREDIT_DAY_OVERDUE'].apply(lambda x: 1 if x > 100 else 0)
                                                                     
# #Debt ratio                                                         
# bureau['BUREAU_CREDIT_DEBT_RATIO'] = bureau['AMT_CREDIT_SUM_DEBT'] / bureau['AMT_CREDIT_SUM']


# #We will now aggreagte over SK_ID_CURR to calculate relevant numeric features 

# aggregations = {
#         'DAYS_CREDIT': ['sum', 'mean', 'std'],
#         'DAYS_CREDIT_ENDDATE': ['sum', 'mean', 'std'],
#         'CREDIT_DAY_OVERDUE': ['min', 'max', 'sum','mean', 'std'],
#         'AMT_CREDIT_MAX_OVERDUE': ['min', 'max', 'sum', 'mean', 'std'],
#         'AMT_CREDIT_SUM': ['min', 'max', 'sum', 'mean', 'std'],
#         'AMT_CREDIT_SUM_DEBT': ['min', 'max', 'sum', 'mean', 'std'],
#         'AMT_CREDIT_SUM_OVERDUE': ['min', 'max', 'sum', 'mean', 'std'],
#         'AMT_CREDIT_SUM_LIMIT': ['min', 'max', 'sum', 'mean', 'std'],
#         'AMT_ANNUITY': ['min', 'max', 'mean', 'sum', 'std'],
#         'CNT_CREDIT_PROLONG': ['sum'],
#         'SK_ID_BUREAU': ['count'],
#         'DAYS_ENDDATE_FACT': ['min', 'max', 'mean', 'std'],
#         'ENDDATE_DIF': ['min', 'max', 'mean', 'std'],
#         'BUREAU_CREDIT_DEBT_RATIO': ['min', 'max', 'mean','std'],
#         'BUREAU_IS_DPD': ['mean', 'sum', 'std'],
#         'BUREAU_IS_DPD_OVER100': ['mean', 'sum', 'std']
# } 
                                                                     
# bureau_num = bureau.groupby("SK_ID_CURR").agg(aggregations).fillna(0)

# #Renaming columns to remove multi indexing
# bureau_num.columns = pd.Index(['BUREAU' + '_' + e[0] + '_' + e[1] for e in bureau_num.columns])

# bureau_num.head()

,BUREAU_DAYS_CREDIT_sum,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_std,BUREAU_DAYS_CREDIT_ENDDATE_sum,BUREAU_DAYS_CREDIT_ENDDATE_mean,BUREAU_DAYS_CREDIT_ENDDATE_std,BUREAU_CREDIT_DAY_OVERDUE_min,BUREAU_CREDIT_DAY_OVERDUE_max,BUREAU_CREDIT_DAY_OVERDUE_sum,BUREAU_CREDIT_DAY_OVERDUE_mean,BUREAU_CREDIT_DAY_OVERDUE_std,BUREAU_AMT_CREDIT_MAX_OVERDUE_min,BUREAU_AMT_CREDIT_MAX_OVERDUE_max,BUREAU_AMT_CREDIT_MAX_OVERDUE_sum,BUREAU_AMT_CREDIT_MAX_OVERDUE_mean,BUREAU_AMT_CREDIT_MAX_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_min,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_std,BUREAU_AMT_CREDIT_SUM_DEBT_min,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_DEBT_std,BUREAU_AMT_CREDIT_SUM_OVERDUE_min,BUREAU_AMT_CREDIT_SUM_OVERDUE_max,BUREAU_AMT_CREDIT_SUM_OVERDUE_sum,BUREAU_AMT_CREDIT_SUM_OVERDUE_mean,BUREAU_AMT_CREDIT_SUM_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_LIMIT_min,BUREAU_AMT_CREDIT_SUM_LIMIT_max,BUREAU_AMT_CREDIT_SUM_LIMIT_sum,BUREAU_AMT_CREDIT_SUM_LIMIT_mean,BUREAU_AMT_CREDIT_SUM_LIMIT_std,BUREAU_AMT_ANNUITY_min,BUREAU_AMT_ANNUITY_max,BUREAU_AMT_ANNUITY_mean,BUREAU_AMT_ANNUITY_sum,BUREAU_AMT_ANNUITY_std,BUREAU_CNT_CREDIT_PROLONG_sum,BUREAU_SK_ID_BUREAU_count,BUREAU_DAYS_ENDDATE_FACT_min,BUREAU_DAYS_ENDDATE_FACT_max,BUREAU_DAYS_ENDDATE_FACT_mean,BUREAU_DAYS_ENDDATE_FACT_std,BUREAU_ENDDATE_DIF_min,BUREAU_ENDDATE_DIF_max,BUREAU_ENDDATE_DIF_mean,BUREAU_ENDDATE_DIF_std,BUREAU_BUREAU_CREDIT_DEBT_RATIO_min,BUREAU_BUREAU_CREDIT_DEBT_RATIO_max,BUREAU_BUREAU_CREDIT_DEBT_RATIO_mean,BUREAU_BUREAU_CREDIT_DEBT_RATIO_std,BUREAU_BUREAU_IS_DPD_mean,BUREAU_BUREAU_IS_DPD_sum,BUREAU_BUREAU_IS_DPD_std,BUREAU_BUREAU_IS_DPD_OVER100_mean,BUREAU_BUREAU_IS_DPD_OVER100_sum,BUREAU_BUREAU_IS_DPD_OVER100_std
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-735.000000,489.942514,577.0,82.428571,1032.859277,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,85500.0,378000.0,1453365.000,207623.571429,122544.544510,0.0,373239.0,596686.5,85240.928571,137485.631124,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.00000,0.0000,0.0,10822.5,3545.357143,24817.5,4800.607529,0,7,-1328.0,-544.0,-825.500000,369.078582,-1.0,698.0,197.0,334.957709,0.0,0.987405,0.282518,0.399523,0.0,0,0.0,0.0,0,0.0
100002,-6992,-874.000000,431.451040,-2094.0,-349.000000,767.490977,0,0,0,0.0,0.0,0.0,5043.645,8405.145,1681.029,2363.2469,0.0,450000.0,865055.565,108131.945625,146075.557435,0.0,245781.0,245781.0,49156.200000,109916.604716,0.0,0.0,0.0,0.0,0.0,0.0,31988.565,31988.565,7997.14125,15994.2825,0.0,0.0,0.000000,0.0,0.000000,0,8,-1185.0,-36.0,-697.500000,515.992539,0.0,1029.0,252.6,437.954678,0.0,0.546180,0.136545,0.273090,0.0,0,0.0,0.0,0,0.0
100003,-5603,-1400.750000,909.826128,-2178.0,-544.500000,1492.770467,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,22248.0,810000.0,1017400.500,254350.125000,372269.465535,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,810000.000,810000.000,202500.00000,405000.0000,0.0,0.0,0.000000,0.0,0.000000,0,4,-2131.0,-540.0,-1097.333333,896.097279,-303.0,201.0,-34.0,253.714406,0.0,0.000000,0.000000,0.000000,0.0,0,0.0,0.0,0,0.0
100004,-1734,-867.000000,649.124025,-977.0,-488.500000,150.613744,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,94500.0,94537.8,189037.800,94518.900000,26.728636,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.00000,0.0000,0.0,0.0,0.000000,0.0,0.000000,0,2,-683.0,-382.0,-532.500000,212.839141,0.0,88.0,44.0,62.225397,0.0,0.000000,0.000000,0.000000,0.0,0,0.0,0.0,0,0.0
100005,-572,-190.666667,162.297053,1318.0,439.333333,776.274007,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,29826.0,568800.0,657126.000,219042.000000,303238.426806,0.0,543087.0,568408.5,189469.500000,306503.339003,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.00000,0.0000,0.0,4261.5,1420.500000,4261.5,2460.378172,0,3,-123.0,-123.0,-123.000000,0.000000,-5.0,-5.0,-5.0,0.000000,0.0,0.954794,0.601256,0.523384,0.0,0,0.0,0.0,0,0.0


In [96]:
# #We will count the number of CB credits with each of these attributes :
# bureau_categ1 = pd.crosstab(bureau['SK_ID_CURR'], bureau['CREDIT_ACTIVE'])
# bureau_categ2 = pd.crosstab(bureau['SK_ID_CURR'], bureau['CREDIT_CURRENCY'])

# bureau_categ = pd.merge(bureau_categ1, bureau_categ2, how="outer", left_index=True, right_index=True)

# cols = ['CB_ACTIVE', 'CB_BAD_DEBT', 'CB_CLOSED', 'CB_SOLD',
#                         'CB_CURR1', 'CB_CURR2', 'CB_CURR3', 'CB_CURR4']
# bureau_categ.columns = cols

# bureau_categ = pd.merge(bureau_categ, bureau_num[["BUREAU_SK_ID_BUREAU_count"]], how="inner", left_index=True, right_index=True)

# for c in cols:
#     bureau_categ[c] = bureau_categ[c] / bureau_categ["BUREAU_SK_ID_BUREAU_count"]
# bureau_categ.head()

,CB_ACTIVE,CB_BAD_DEBT,CB_CLOSED,CB_SOLD,CB_CURR1,CB_CURR2,CB_CURR3,CB_CURR4,BUREAU_SK_ID_BUREAU_count
SK_ID_CURR,,,,,,,,,
100001,0.428571,0.0,0.571429,0.0,1.0,0.0,0.0,0.0,7
100002,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,8
100003,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,4
100004,0.000000,0.0,1.000000,0.0,1.0,0.0,0.0,0.0,2
100005,0.666667,0.0,0.333333,0.0,1.0,0.0,0.0,0.0,3


In [98]:
# #We now load the bureau_balance csv file
# bureau_balance = pd.read_csv("Data/bureau_balance.csv", sep=',')

# bureau_balance.STATUS.value_counts(normalize=True)

# bureau_balance.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [105]:
# #We create a crosstab to count the number of status type for each sk_id_bureau
# bureau_balance_stats = pd.crosstab(bureau_balance['SK_ID_BUREAU'], bureau_balance['STATUS'])

# bureau_balance_stats.head()

STATUS,0,1,2,3,4,5,C,X
SK_ID_BUREAU,,,,,,,,
5001709,0,0,0,0,0,0,86,11
5001710,5,0,0,0,0,0,48,30
5001711,3,0,0,0,0,0,0,1
5001712,10,0,0,0,0,0,9,0
5001713,0,0,0,0,0,0,0,22


In [106]:
# #Counting the number of columns for each sk_id_bureau
# bureau_balance_count = bureau_balance[["SK_ID_BUREAU","MONTHS_BALANCE"]].groupby("SK_ID_BUREAU").count()

# bureau_balance_count.columns = ["CB_COUNT"]

# #Renaming the columns for better clarity
# cols = ["CB_DPD_0","CB_DPD_1","CB_DPD_2","CB_DPD_3","CB_DPD_4","CB_DPD_5","CB_BAL_CLOSED","CB_BAL_UKN"]
# bureau_balance_stats.columns=["CB_DPD_0","CB_DPD_1","CB_DPD_2","CB_DPD_3","CB_DPD_4","CB_DPD_5","CB_BAL_CLOSED","CB_BAL_UKN"]

# bureau_balance_stats = pd.merge(bureau_balance_stats, bureau_balance_count, how="inner", left_index=True, right_index=True)

# for c in cols:
#     bureau_balance_stats[c] = bureau_balance_stats[c] / bureau_balance_stats["CB_COUNT"]

# bureau_balance_stats.head()

,CB_DPD_0,CB_DPD_1,CB_DPD_2,CB_DPD_3,CB_DPD_4,CB_DPD_5,CB_BAL_CLOSED,CB_BAL_UKN,CB_COUNT
SK_ID_BUREAU,,,,,,,,,
5001709,0.000000,0.0,0.0,0.0,0.0,0.0,0.886598,0.113402,97
5001710,0.060241,0.0,0.0,0.0,0.0,0.0,0.578313,0.361446,83
5001711,0.750000,0.0,0.0,0.0,0.0,0.0,0.000000,0.250000,4
5001712,0.526316,0.0,0.0,0.0,0.0,0.0,0.473684,0.000000,19
5001713,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,22


In [111]:
# #Joining with the main CB dataframe to retrieve SK_ID_CURR info
# bureau_num_bal = pd.merge(bureau_balance_stats, bureau[["SK_ID_BUREAU","SK_ID_CURR"]], how='inner', left_index=True, right_on='SK_ID_BUREAU')

# #Creating aggregator
# agg_functions = ['min', 'max', 'mean', 'std', 'sum']

# aggregations = {
#     c: agg_functions for c in bureau_balance_stats.columns
# }

# #Aggregating by SK_ID_CURR
# bureau_num_bal = bureau_num_bal.groupby("SK_ID_CURR").agg(aggregations)

# #Renaming columns to remove multi indexing
# bureau_num_bal.columns = pd.Index(['BB' + '_' + e[0] + '_' + e[1] for e in bureau_num_bal.columns])

# bureau_num_bal.head()

,BB_CB_DPD_0_min,BB_CB_DPD_0_max,BB_CB_DPD_0_mean,BB_CB_DPD_0_std,BB_CB_DPD_0_sum,BB_CB_DPD_1_min,BB_CB_DPD_1_max,BB_CB_DPD_1_mean,BB_CB_DPD_1_std,BB_CB_DPD_1_sum,BB_CB_DPD_2_min,BB_CB_DPD_2_max,BB_CB_DPD_2_mean,BB_CB_DPD_2_std,BB_CB_DPD_2_sum,BB_CB_DPD_3_min,BB_CB_DPD_3_max,BB_CB_DPD_3_mean,BB_CB_DPD_3_std,BB_CB_DPD_3_sum,BB_CB_DPD_4_min,BB_CB_DPD_4_max,BB_CB_DPD_4_mean,BB_CB_DPD_4_std,BB_CB_DPD_4_sum,BB_CB_DPD_5_min,BB_CB_DPD_5_max,BB_CB_DPD_5_mean,BB_CB_DPD_5_std,BB_CB_DPD_5_sum,BB_CB_BAL_CLOSED_min,BB_CB_BAL_CLOSED_max,BB_CB_BAL_CLOSED_mean,BB_CB_BAL_CLOSED_std,BB_CB_BAL_CLOSED_sum,BB_CB_BAL_UKN_min,BB_CB_BAL_UKN_max,BB_CB_BAL_UKN_mean,BB_CB_BAL_UKN_std,BB_CB_BAL_UKN_sum,BB_CB_COUNT_min,BB_CB_COUNT_max,BB_CB_COUNT_mean,BB_CB_COUNT_std,BB_CB_COUNT_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.019231,1.000000,0.336651,0.381334,2.356557,0.0,0.052632,0.007519,0.019893,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.966667,0.441240,0.428578,3.088683,0.0,0.500000,0.214590,0.182611,1.502129,2,52,24.571429,16.050515,172
100002,0.187500,0.818182,0.406960,0.196494,3.255682,0.0,0.500000,0.255682,0.204094,2.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.812500,0.175426,0.263147,1.403409,0.0,0.500000,0.161932,0.161650,1.295455,4,22,13.750000,6.363961,110
100005,0.538462,1.000000,0.735043,0.238245,2.205128,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.384615,0.128205,0.222058,0.384615,0.0,0.333333,0.136752,0.174535,0.410256,3,13,7.000000,5.291503,21
100010,0.277778,0.277778,0.277778,0.000000,0.555556,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.722222,0.722222,0.722222,0.000000,1.444444,0.0,0.000000,0.000000,0.000000,0.000000,36,36,36.000000,0.000000,72
100013,0.000000,0.618182,0.320718,0.255323,1.282872,0.0,0.045455,0.027701,0.019657,0.110804,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.666667,0.397036,0.302258,1.588142,0.0,1.000000,0.254545,0.497044,1.018182,40,69,57.500000,13.127579,230


In [ ]:
# bureau_num_bal.info()
# #We only have 134k different SK_ID, which is about 40% of our dataset. 
# #We will fill nulls with 0 because it means that the other SK_ID were not referenced at the Credit Bureau

In [112]:
# #Filling nulls with 0 as mentionned previously
# bureau_num_full = pd.merge(bureau_num, bureau_num_bal, how='outer', left_index=True, right_index=True).fillna(0)

# bureau_num_full.info()
# bureau_num_full.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305811 entries, 100001 to 456255
Data columns (total 106 columns):
 #    Column                                Non-Null Count   Dtype  
---   ------                                --------------   -----  
 0    BUREAU_DAYS_CREDIT_sum                305811 non-null  int64  
 1    BUREAU_DAYS_CREDIT_mean               305811 non-null  float64
 2    BUREAU_DAYS_CREDIT_std                305811 non-null  float64
 3    BUREAU_DAYS_CREDIT_ENDDATE_sum        305811 non-null  float64
 4    BUREAU_DAYS_CREDIT_ENDDATE_mean       305811 non-null  float64
 5    BUREAU_DAYS_CREDIT_ENDDATE_std        305811 non-null  float64
 6    BUREAU_CREDIT_DAY_OVERDUE_min         305811 non-null  int64  
 7    BUREAU_CREDIT_DAY_OVERDUE_max         305811 non-null  int64  
 8    BUREAU_CREDIT_DAY_OVERDUE_sum         305811 non-null  int64  
 9    BUREAU_CREDIT_DAY_OVERDUE_mean        305811 non-null  float64
 10   BUREAU_CREDIT_DAY_OVERDUE_std         305811 non-

,BUREAU_DAYS_CREDIT_sum,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_std,BUREAU_DAYS_CREDIT_ENDDATE_sum,BUREAU_DAYS_CREDIT_ENDDATE_mean,BUREAU_DAYS_CREDIT_ENDDATE_std,BUREAU_CREDIT_DAY_OVERDUE_min,BUREAU_CREDIT_DAY_OVERDUE_max,BUREAU_CREDIT_DAY_OVERDUE_sum,BUREAU_CREDIT_DAY_OVERDUE_mean,BUREAU_CREDIT_DAY_OVERDUE_std,BUREAU_AMT_CREDIT_MAX_OVERDUE_min,BUREAU_AMT_CREDIT_MAX_OVERDUE_max,BUREAU_AMT_CREDIT_MAX_OVERDUE_sum,BUREAU_AMT_CREDIT_MAX_OVERDUE_mean,BUREAU_AMT_CREDIT_MAX_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_min,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_std,BUREAU_AMT_CREDIT_SUM_DEBT_min,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_DEBT_std,BUREAU_AMT_CREDIT_SUM_OVERDUE_min,BUREAU_AMT_CREDIT_SUM_OVERDUE_max,BUREAU_AMT_CREDIT_SUM_OVERDUE_sum,BUREAU_AMT_CREDIT_SUM_OVERDUE_mean,BUREAU_AMT_CREDIT_SUM_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_LIMIT_min,BUREAU_AMT_CREDIT_SUM_LIMIT_max,BUREAU_AMT_CREDIT_SUM_LIMIT_sum,BUREAU_AMT_CREDIT_SUM_LIMIT_mean,BUREAU_AMT_CREDIT_SUM_LIMIT_std,BUREAU_AMT_ANNUITY_min,BUREAU_AMT_ANNUITY_max,BUREAU_AMT_ANNUITY_mean,BUREAU_AMT_ANNUITY_sum,BUREAU_AMT_ANNUITY_std,BUREAU_CNT_CREDIT_PROLONG_sum,BUREAU_SK_ID_BUREAU_count,BUREAU_DAYS_ENDDATE_FACT_min,BUREAU_DAYS_ENDDATE_FACT_max,BUREAU_DAYS_ENDDATE_FACT_mean,BUREAU_DAYS_ENDDATE_FACT_std,BUREAU_ENDDATE_DIF_min,BUREAU_ENDDATE_DIF_max,BUREAU_ENDDATE_DIF_mean,...,BUREAU_BUREAU_IS_DPD_sum,BUREAU_BUREAU_IS_DPD_std,BUREAU_BUREAU_IS_DPD_OVER100_mean,BUREAU_BUREAU_IS_DPD_OVER100_sum,BUREAU_BUREAU_IS_DPD_OVER100_std,BB_CB_DPD_0_min,BB_CB_DPD_0_max,BB_CB_DPD_0_mean,BB_CB_DPD_0_std,BB_CB_DPD_0_sum,BB_CB_DPD_1_min,BB_CB_DPD_1_max,BB_CB_DPD_1_mean,BB_CB_DPD_1_std,BB_CB_DPD_1_sum,BB_CB_DPD_2_min,BB_CB_DPD_2_max,BB_CB_DPD_2_mean,BB_CB_DPD_2_std,BB_CB_DPD_2_sum,BB_CB_DPD_3_min,BB_CB_DPD_3_max,BB_CB_DPD_3_mean,BB_CB_DPD_3_std,BB_CB_DPD_3_sum,BB_CB_DPD_4_min,BB_CB_DPD_4_max,BB_CB_DPD_4_mean,BB_CB_DPD_4_std,BB_CB_DPD_4_sum,BB_CB_DPD_5_min,BB_CB_DPD_5_max,BB_CB_DPD_5_mean,BB_CB_DPD_5_std,BB_CB_DPD_5_sum,BB_CB_BAL_CLOSED_min,BB_CB_BAL_CLOSED_max,BB_CB_BAL_CLOSED_mean,BB_CB_BAL_CLOSED_std,BB_CB_BAL_CLOSED_sum,BB_CB_BAL_UKN_min,BB_CB_BAL_UKN_max,BB_CB_BAL_UKN_mean,BB_CB_BAL_UKN_std,BB_CB_BAL_UKN_sum,BB_CB_COUNT_min,BB_CB_COUNT_max,BB_CB_COUNT_mean,BB_CB_COUNT_std,BB_CB_COUNT_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-5145,-735.000000,489.942514,577.0,82.428571,1032.859277,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,85500.0,378000.0,1453365.000,207623.571429,122544.544510,0.0,373239.0,596686.5,85240.928571,137485.631124,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.00000,0.0000,0.0,10822.5,3545.357143,24817.5,4800.607529,0,7,-1328.0,-544.0,-825.500000,369.078582,-1.0,698.0,197.0,...,0,0.0,0.0,0,0.0,0.019231,1.000000,0.336651,0.381334,2.356557,0.0,0.052632,0.007519,0.019893,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.966667,0.441240,0.428578,3.088683,0.0,0.500000,0.214590,0.182611,1.502129,2.0,52.0,24.571429,16.050515,172.0
100002,-6992,-874.000000,431.451040,-2094.0,-349.000000,767.490977,0,0,0,0.0,0.0,0.0,5043.645,8405.145,1681.029,2363.2469,0.0,450000.0,865055.565,108131.945625,146075.557435,0.0,245781.0,245781.0,49156.200000,109916.604716,0.0,0.0,0.0,0.0,0.0,0.0,31988.565,31988.565,7997.14125,15994.2825,0.0,0.0,0.000000,0.0,0.000000,0,8,-1185.0,-36.0,-697.500000,515.992539,0.0,1029.0,252.6,...,0,0.0,0.0,0,0.0,0.187500,0.818182,0.406960,0.196494,3.255682,0.0,0.500000,0.255682,0.204094,2.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.812500,0.175426,0.263147,1.403409,0.0,0.500000,0.161932,0.161650,1.295455,4.0,22.0,13.750000,6.363961,110.0
100003,-5603,-1400.750000,909.826128,-2178.0,-544.500000,1492.770467,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,22248.0,810000.0,1017400.500,254350.125000,372269.

In [113]:
# #Merging the 2 dataframes with bureau information
# bureau_df = pd.merge(bureau_categ, bureau_num_full, how='outer', left_index=True, right_index=True)

# bureau_df.head()

0


,CB_ACTIVE,CB_BAD_DEBT,CB_CLOSED,CB_SOLD,CB_CURR1,CB_CURR2,CB_CURR3,CB_CURR4,BUREAU_SK_ID_BUREAU_count_x,BUREAU_DAYS_CREDIT_sum,BUREAU_DAYS_CREDIT_mean,BUREAU_DAYS_CREDIT_std,BUREAU_DAYS_CREDIT_ENDDATE_sum,BUREAU_DAYS_CREDIT_ENDDATE_mean,BUREAU_DAYS_CREDIT_ENDDATE_std,BUREAU_CREDIT_DAY_OVERDUE_min,BUREAU_CREDIT_DAY_OVERDUE_max,BUREAU_CREDIT_DAY_OVERDUE_sum,BUREAU_CREDIT_DAY_OVERDUE_mean,BUREAU_CREDIT_DAY_OVERDUE_std,BUREAU_AMT_CREDIT_MAX_OVERDUE_min,BUREAU_AMT_CREDIT_MAX_OVERDUE_max,BUREAU_AMT_CREDIT_MAX_OVERDUE_sum,BUREAU_AMT_CREDIT_MAX_OVERDUE_mean,BUREAU_AMT_CREDIT_MAX_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_min,BUREAU_AMT_CREDIT_SUM_max,BUREAU_AMT_CREDIT_SUM_sum,BUREAU_AMT_CREDIT_SUM_mean,BUREAU_AMT_CREDIT_SUM_std,BUREAU_AMT_CREDIT_SUM_DEBT_min,BUREAU_AMT_CREDIT_SUM_DEBT_max,BUREAU_AMT_CREDIT_SUM_DEBT_sum,BUREAU_AMT_CREDIT_SUM_DEBT_mean,BUREAU_AMT_CREDIT_SUM_DEBT_std,BUREAU_AMT_CREDIT_SUM_OVERDUE_min,BUREAU_AMT_CREDIT_SUM_OVERDUE_max,BUREAU_AMT_CREDIT_SUM_OVERDUE_sum,BUREAU_AMT_CREDIT_SUM_OVERDUE_mean,BUREAU_AMT_CREDIT_SUM_OVERDUE_std,BUREAU_AMT_CREDIT_SUM_LIMIT_min,BUREAU_AMT_CREDIT_SUM_LIMIT_max,BUREAU_AMT_CREDIT_SUM_LIMIT_sum,BUREAU_AMT_CREDIT_SUM_LIMIT_mean,BUREAU_AMT_CREDIT_SUM_LIMIT_std,BUREAU_AMT_ANNUITY_min,BUREAU_AMT_ANNUITY_max,BUREAU_AMT_ANNUITY_mean,BUREAU_AMT_ANNUITY_sum,BUREAU_AMT_ANNUITY_std,...,BUREAU_BUREAU_IS_DPD_sum,BUREAU_BUREAU_IS_DPD_std,BUREAU_BUREAU_IS_DPD_OVER100_mean,BUREAU_BUREAU_IS_DPD_OVER100_sum,BUREAU_BUREAU_IS_DPD_OVER100_std,BB_CB_DPD_0_min,BB_CB_DPD_0_max,BB_CB_DPD_0_mean,BB_CB_DPD_0_std,BB_CB_DPD_0_sum,BB_CB_DPD_1_min,BB_CB_DPD_1_max,BB_CB_DPD_1_mean,BB_CB_DPD_1_std,BB_CB_DPD_1_sum,BB_CB_DPD_2_min,BB_CB_DPD_2_max,BB_CB_DPD_2_mean,BB_CB_DPD_2_std,BB_CB_DPD_2_sum,BB_CB_DPD_3_min,BB_CB_DPD_3_max,BB_CB_DPD_3_mean,BB_CB_DPD_3_std,BB_CB_DPD_3_sum,BB_CB_DPD_4_min,BB_CB_DPD_4_max,BB_CB_DPD_4_mean,BB_CB_DPD_4_std,BB_CB_DPD_4_sum,BB_CB_DPD_5_min,BB_CB_DPD_5_max,BB_CB_DPD_5_mean,BB_CB_DPD_5_std,BB_CB_DPD_5_sum,BB_CB_BAL_CLOSED_min,BB_CB_BAL_CLOSED_max,BB_CB_BAL_CLOSED_mean,BB_CB_BAL_CLOSED_std,BB_CB_BAL_CLOSED_sum,BB_CB_BAL_UKN_min,BB_CB_BAL_UKN_max,BB_CB_BAL_UKN_mean,BB_CB_BAL_UKN_std,BB_CB_BAL_UKN_sum,BB_CB_COUNT_min,BB_CB_COUNT_max,BB_CB_COUNT_mean,BB_CB_COUNT_std,BB_CB_COUNT_sum
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,0.428571,0.0,0.571429,0.0,1.0,0.0,0.0,0.0,7,-5145,-735.000000,489.942514,577.0,82.428571,1032.859277,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,85500.0,378000.0,1453365.000,207623.571429,122544.544510,0.0,373239.0,596686.5,85240.928571,137485.631124,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.00000,0.0000,0.0,10822.5,3545.357143,24817.5,4800.607529,...,0,0.0,0.0,0,0.0,0.019231,1.000000,0.336651,0.381334,2.356557,0.0,0.052632,0.007519,0.019893,0.052632,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.966667,0.441240,0.428578,3.088683,0.0,0.500000,0.214590,0.182611,1.502129,2.0,52.0,24.571429,16.050515,172.0
100002,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,8,-6992,-874.000000,431.451040,-2094.0,-349.000000,767.490977,0,0,0,0.0,0.0,0.0,5043.645,8405.145,1681.029,2363.2469,0.0,450000.0,865055.565,108131.945625,146075.557435,0.0,245781.0,245781.0,49156.200000,109916.604716,0.0,0.0,0.0,0.0,0.0,0.0,31988.565,31988.565,7997.14125,15994.2825,0.0,0.0,0.000000,0.0,0.000000,...,0,0.0,0.0,0,0.0,0.187500,0.818182,0.406960,0.196494,3.255682,0.0,0.500000,0.255682,0.204094,2.045455,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.812500,0.175426,0.263147,1.403409,0.0,0.500000,0.161932,0.161650,1.295455,4.0,22.0,13.750000,6.363961,110.0
100003,0.250000,0.0,0.750000,0.0,1.0,0.0,0.0,0.0,4,-5603,-1400.750000,909.826128,-2178.0,-544.500000,1492.770467,0,0,0,0.0,0.0,0.0,0.000,0.000,0.000,0.0000,22248.0,810000.0,1017400.500,254350.125000,372269.465535,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,810000.000,810000.000,202500.00000,405000.0000,0.0,0.0,0.000000

In [114]:
# #Saving bureau_df to reduce RAM usage
# bureau_df.to_csv("Data/bureau_df.csv")

In [115]:
def load_bureau_info(df, X_test, X_val=None, validation_set=False):

    #Loading the csv file
    bureau_df = pd.read_csv("Data/bureau_df.csv")

    #Joining this new data and filling NAs with 0
    df = pd.merge(df, bureau_df, how='left', left_index=True, right_index=True).fillna(0)
    X_test = pd.merge(X_test, bureau_df, how='left', left_index=True, right_index=True).fillna(0)
    if validation_set:
        X_val = pd.merge(X_val, bureau_df, how='left', left_index=True, right_index=True).fillna(0)

### 3.5 Using Cash balance information

In [123]:
# cash = pd.read_csv("Data/POS_CASH_balance.csv", sep=',')

# cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


In [124]:
# #Creating late payment flag
# cash['LATE_PAYMENT'] = cash['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
# cash['POS_IS_DPD_OVER_100'] = cash['SK_DPD'].apply(lambda x: 1 if x >= 100 else 0)

# #Creating aggregator
# agg_functions = ['min', 'max', 'mean', 'std', 'sum']

# #Aggregating on all columns except months_balance and sk_id_prev
# aggregations = {
#     c: agg_functions for c in cash.drop(columns={"SK_ID_PREV","MONTHS_BALANCE", "NAME_CONTRACT_STATUS"}).columns
# }

# #Adding a nunique count on SK_ID_PREV
# aggregations["SK_ID_PREV"] = "nunique"

# #Aggregating over "SK_ID_CURR"
# cash_df = cash.groupby("SK_ID_CURR").agg(aggregations).fillna(0)

# #Renaming columns to remove multi indexing
# cash_df.columns = pd.Index(['CASH' + '_' + e[0] + '_' + e[1] for e in cash_df.columns])

# cash_df.head()

,CASH_SK_ID_CURR_min,CASH_SK_ID_CURR_max,CASH_SK_ID_CURR_mean,CASH_SK_ID_CURR_std,CASH_SK_ID_CURR_sum,CASH_CNT_INSTALMENT_min,CASH_CNT_INSTALMENT_max,CASH_CNT_INSTALMENT_mean,CASH_CNT_INSTALMENT_std,CASH_CNT_INSTALMENT_sum,CASH_CNT_INSTALMENT_FUTURE_min,CASH_CNT_INSTALMENT_FUTURE_max,CASH_CNT_INSTALMENT_FUTURE_mean,CASH_CNT_INSTALMENT_FUTURE_std,CASH_CNT_INSTALMENT_FUTURE_sum,CASH_SK_DPD_min,CASH_SK_DPD_max,CASH_SK_DPD_mean,CASH_SK_DPD_std,CASH_SK_DPD_sum,CASH_SK_DPD_DEF_min,CASH_SK_DPD_DEF_max,CASH_SK_DPD_DEF_mean,CASH_SK_DPD_DEF_std,CASH_SK_DPD_DEF_sum,CASH_LATE_PAYMENT_min,CASH_LATE_PAYMENT_max,CASH_LATE_PAYMENT_mean,CASH_LATE_PAYMENT_std,CASH_LATE_PAYMENT_sum,CASH_POS_IS_DPD_OVER_100_min,CASH_POS_IS_DPD_OVER_100_max,CASH_POS_IS_DPD_OVER_100_mean,CASH_POS_IS_DPD_OVER_100_std,CASH_POS_IS_DPD_OVER_100_sum,CASH_SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,100001.0,0.0,900009,4.0,4.0,4.000000,0.000000,36.0,0.0,4.0,1.444444,1.424001,13.0,0,7,0.777778,2.333333,7,0,7,0.777778,2.333333,7,0,1,0.111111,0.333333,1,0,0,0.0,0.0,0,2
100002,100002,100002,100002.0,0.0,1900038,24.0,24.0,24.000000,0.000000,456.0,6.0,24.0,15.000000,5.627314,285.0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.0,0.0,0,1
100003,100003,100003,100003.0,0.0,2800084,6.0,12.0,10.107143,2.806597,283.0,0.0,12.0,5.785714,3.842811,162.0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.0,0.0,0,3
100004,100004,100004,100004.0,0.0,400016,3.0,4.0,3.750000,0.500000,15.0,0.0,4.0,2.250000,1.707825,9.0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.0,0.0,0,1
100005,100005,100005,100005.0,0.0,1100055,9.0,12.0,11.700000,0.948683,117.0,0.0,12.0,7.200000,3.614784,72.0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.0,0.0,0,1


In [125]:
# #Saving cash_df to csv to save RAM usage
# cash_df.to_csv("Data/cash_df.csv")

In [121]:
def load_cash_info(df, X_test, X_val=None, validation_set=False):

    #Loading the csv file
    cash_df = pd.read_csv("Data/cash_df.csv")

    #Joining this new data and filling NAs with 0
    df = pd.merge(df, cash_df, how='left', left_index=True, right_index=True).fillna(0)
    X_test = pd.merge(X_test, cash_df, how='left', left_index=True, right_index=True).fillna(0)
    if validation_set:
        X_val = pd.merge(X_val, cash_df, how='left', left_index=True, right_index=True).fillna(0)


### 3.6 Using CC Balance information

In [122]:
# cc = pd.read_csv("Data/credit_card_balance.csv",sep=",")

# cc.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [126]:
# #Calculating new features
# cc['LATE_PAYMENT'] = cc['SK_DPD'].apply(lambda x: 1 if x > 0 else 0)
# cc['DRAWING_LIMIT_RATIO'] = cc['AMT_DRAWINGS_ATM_CURRENT'] / cc['AMT_CREDIT_LIMIT_ACTUAL']
# cc['CARD_IS_DPD_OVER_100'] = cc['SK_DPD'].apply(lambda x: 1 if x >= 100 else 0)

# #Creating aggregator
# agg_functions = ['min', 'max', 'mean', 'std', 'sum']

# #Aggregating on all columns except months_balance and sk_id_prev
# aggregations = {
#     c: agg_functions for c in cc.drop(columns={"SK_ID_PREV","MONTHS_BALANCE", "NAME_CONTRACT_STATUS"}).columns
# }

# #Adding a nunique count on SK_ID_PREV
# aggregations["SK_ID_PREV"] = "nunique"

# #Aggregating over "SK_ID_CURR"
# cc_df = cc.groupby("SK_ID_CURR").agg(aggregations).fillna(0)

# #Renaming columns to remove multi indexing
# cc_df.columns = pd.Index(['CC' + '_' + e[0] + '_' + e[1] for e in cc_df.columns])

# cc_df.head()

# # #Investigating possible months balance values
# # cc.MONTHS_BALANCE.value_counts()

,CC_SK_ID_CURR_min,CC_SK_ID_CURR_max,CC_SK_ID_CURR_mean,CC_SK_ID_CURR_std,CC_SK_ID_CURR_sum,CC_AMT_BALANCE_min,CC_AMT_BALANCE_max,CC_AMT_BALANCE_mean,CC_AMT_BALANCE_std,CC_AMT_BALANCE_sum,CC_AMT_CREDIT_LIMIT_ACTUAL_min,CC_AMT_CREDIT_LIMIT_ACTUAL_max,CC_AMT_CREDIT_LIMIT_ACTUAL_mean,CC_AMT_CREDIT_LIMIT_ACTUAL_std,CC_AMT_CREDIT_LIMIT_ACTUAL_sum,CC_AMT_DRAWINGS_ATM_CURRENT_min,CC_AMT_DRAWINGS_ATM_CURRENT_max,CC_AMT_DRAWINGS_ATM_CURRENT_mean,CC_AMT_DRAWINGS_ATM_CURRENT_std,CC_AMT_DRAWINGS_ATM_CURRENT_sum,CC_AMT_DRAWINGS_CURRENT_min,CC_AMT_DRAWINGS_CURRENT_max,CC_AMT_DRAWINGS_CURRENT_mean,CC_AMT_DRAWINGS_CURRENT_std,CC_AMT_DRAWINGS_CURRENT_sum,CC_AMT_DRAWINGS_OTHER_CURRENT_min,CC_AMT_DRAWINGS_OTHER_CURRENT_max,CC_AMT_DRAWINGS_OTHER_CURRENT_mean,CC_AMT_DRAWINGS_OTHER_CURRENT_std,CC_AMT_DRAWINGS_OTHER_CURRENT_sum,CC_AMT_DRAWINGS_POS_CURRENT_min,CC_AMT_DRAWINGS_POS_CURRENT_max,CC_AMT_DRAWINGS_POS_CURRENT_mean,CC_AMT_DRAWINGS_POS_CURRENT_std,CC_AMT_DRAWINGS_POS_CURRENT_sum,CC_AMT_INST_MIN_REGULARITY_min,CC_AMT_INST_MIN_REGULARITY_max,CC_AMT_INST_MIN_REGULARITY_mean,CC_AMT_INST_MIN_REGULARITY_std,CC_AMT_INST_MIN_REGULARITY_sum,CC_AMT_PAYMENT_CURRENT_min,CC_AMT_PAYMENT_CURRENT_max,CC_AMT_PAYMENT_CURRENT_mean,CC_AMT_PAYMENT_CURRENT_std,CC_AMT_PAYMENT_CURRENT_sum,CC_AMT_PAYMENT_TOTAL_CURRENT_min,CC_AMT_PAYMENT_TOTAL_CURRENT_max,CC_AMT_PAYMENT_TOTAL_CURRENT_mean,CC_AMT_PAYMENT_TOTAL_CURRENT_std,CC_AMT_PAYMENT_TOTAL_CURRENT_sum,...,CC_CNT_DRAWINGS_ATM_CURRENT_max,CC_CNT_DRAWINGS_ATM_CURRENT_mean,CC_CNT_DRAWINGS_ATM_CURRENT_std,CC_CNT_DRAWINGS_ATM_CURRENT_sum,CC_CNT_DRAWINGS_CURRENT_min,CC_CNT_DRAWINGS_CURRENT_max,CC_CNT_DRAWINGS_CURRENT_mean,CC_CNT_DRAWINGS_CURRENT_std,CC_CNT_DRAWINGS_CURRENT_sum,CC_CNT_DRAWINGS_OTHER_CURRENT_min,CC_CNT_DRAWINGS_OTHER_CURRENT_max,CC_CNT_DRAWINGS_OTHER_CURRENT_mean,CC_CNT_DRAWINGS_OTHER_CURRENT_std,CC_CNT_DRAWINGS_OTHER_CURRENT_sum,CC_CNT_DRAWINGS_POS_CURRENT_min,CC_CNT_DRAWINGS_POS_CURRENT_max,CC_CNT_DRAWINGS_POS_CURRENT_mean,CC_CNT_DRAWINGS_POS_CURRENT_std,CC_CNT_DRAWINGS_POS_CURRENT_sum,CC_CNT_INSTALMENT_MATURE_CUM_min,CC_CNT_INSTALMENT_MATURE_CUM_max,CC_CNT_INSTALMENT_MATURE_CUM_mean,CC_CNT_INSTALMENT_MATURE_CUM_std,CC_CNT_INSTALMENT_MATURE_CUM_sum,CC_SK_DPD_min,CC_SK_DPD_max,CC_SK_DPD_mean,CC_SK_DPD_std,CC_SK_DPD_sum,CC_SK_DPD_DEF_min,CC_SK_DPD_DEF_max,CC_SK_DPD_DEF_mean,CC_SK_DPD_DEF_std,CC_SK_DPD_DEF_sum,CC_LATE_PAYMENT_min,CC_LATE_PAYMENT_max,CC_LATE_PAYMENT_mean,CC_LATE_PAYMENT_std,CC_LATE_PAYMENT_sum,CC_DRAWING_LIMIT_RATIO_min,CC_DRAWING_LIMIT_RATIO_max,CC_DRAWING_LIMIT_RATIO_mean,CC_DRAWING_LIMIT_RATIO_std,CC_DRAWING_LIMIT_RATIO_sum,CC_CARD_IS_DPD_OVER_100_min,CC_CARD_IS_DPD_OVER_100_max,CC_CARD_IS_DPD_OVER_100_mean,CC_CARD_IS_DPD_OVER_100_std,CC_CARD_IS_DPD_OVER_100_sum,CC_SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100006,100006,100006,100006.0,0.0,600036,0.0,0.00,0.000000,0.000000,0.000,270000,270000,270000.000000,0.000000,1620000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000,0.0,0.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.000,...,0.0,0.000000,0.000000,0.0,0,0,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0,0,0.000000,0.000000,0,0.0,0.0,0.000000,0.000000,0.000000,0,0,0.0,0.0,0,1
100011,100011,100011,100011.0,0.0,7400814,0.0,189000.00,54482.111149,68127.238270,4031676.225,90000,180000,164189.189189,34482.743620,12150000,0.0,180000.0,2432.432432,20924.574974,180000.0,0.0,180000.0,2432.432432,20924.574974,180000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9000.0,3956.221849,4487.750710,288804.195,0.0,55485.0,4843.064189,7279.601955,358386.75,0.0,55485.0,4520.067568,7473.872687,334485.000,...,4.0,0.054054,0.464991,4.0,0,4,0.054054,0.464991,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,33.0,25.767123,10.288236,1881.0,0,0,0.00000

In [127]:
# #Saving cc_df to prevent high RAM usage
# cc_df.to_csv("Data/cc_df.csv")

In [128]:
def load_cc_info(df, X_test, X_val=None, validation_set=False):

    #Loading the data
    cc_df = pd.read_csv("Data/cc_df.csv")

    #Joining this new data and filling NAs with 0
    df = pd.merge(df, cc_df, how='left', left_index=True, right_index=True).fillna(0)
    X_test = pd.merge(X_test, cc_df, how='left', left_index=True, right_index=True).fillna(0)
    if validation_set:
        X_val = pd.merge(X_val, cc_df, how='left', left_index=True, right_index=True).fillna(0)

### 3.7 Using installment payments information

In [129]:
# install = pd.read_csv("Data/installments_payments.csv",sep=",")

# install.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


In [131]:
# #Converting the DAYS columns into positive values
# install["DAYS_INSTALMENT"] = install["DAYS_INSTALMENT"].apply(lambda x: abs(x))
# install["DAYS_ENTRY_PAYMENT"] = install["DAYS_ENTRY_PAYMENT"].apply(lambda x: abs(x))

# #Calculating simple differences
# install["DAYS_DELAY"] = install["DAYS_ENTRY_PAYMENT"] - install["DAYS_INSTALMENT"]
# install['PAID_OVER_AMOUNT'] = install['AMT_PAYMENT'] - install['AMT_INSTALMENT']

# #PAID_OVER flag
# install['PAID_OVER'] = install['PAID_OVER_AMOUNT'].apply(lambda x: 1 if x > 0 else 0)

# #Difference between DPD and Days before due
# install['DPD_diff'] = install['DAYS_ENTRY_PAYMENT'] - install['DAYS_INSTALMENT']
# install['DBD_diff'] = install['DAYS_INSTALMENT'] - install['DAYS_ENTRY_PAYMENT']

# #Late payment ratio
# install['LATE_PAYMENT'] = install.apply(lambda x: 1 if x['DPD_diff'] > 0 else 0, axis=1)
# install['INSTALMENT_PAYMENT_RATIO'] = install['AMT_PAYMENT'] / install['AMT_INSTALMENT']
# install['LATE_PAYMENT_RATIO'] = install.apply(lambda x: x['INSTALMENT_PAYMENT_RATIO'] if x['LATE_PAYMENT'] == 1 else 0, axis=1)

# #Over 100 flag
# install['INS_IS_DPD_OVER_100'] = install['DPD_diff'].apply(lambda x: 1 if (x >= 100) else 0)

# #We have both high negative and positive delay values which indicate early or very late payment
# #We will now calculate the difference in percentage between AMT_INSTALMENT and AMT_PAYMENT
# install["DEFICIT_PCT"] = (install["AMT_INSTALMENT"] - install["AMT_PAYMENT"])*100/install["AMT_INSTALMENT"]

# install.head()

NameError: name 'ins' is not defined

In [134]:
# #Creating aggregator
# agg_functions = ['min', 'max', 'mean', 'std', 'sum']

# #Aggregating on all columns except sk_id_prev
# aggregations = {
#     c: agg_functions for c in install.drop(columns={"SK_ID_PREV"}).columns
# }

# #Adding a nunique count on SK_ID_PREV
# aggregations["SK_ID_PREV"] = "nunique"

# #Aggregating over "SK_ID_CURR"
# install_df = install.groupby("SK_ID_CURR").agg(aggregations).fillna(0)

# #Renaming columns to remove multi indexing
# install_df.columns = pd.Index(['CC' + '_' + e[0] + '_' + e[1] for e in install_df.columns])

# install_df.head()

,CC_SK_ID_CURR_min,CC_SK_ID_CURR_max,CC_SK_ID_CURR_mean,CC_SK_ID_CURR_std,CC_SK_ID_CURR_sum,CC_NUM_INSTALMENT_VERSION_min,CC_NUM_INSTALMENT_VERSION_max,CC_NUM_INSTALMENT_VERSION_mean,CC_NUM_INSTALMENT_VERSION_std,CC_NUM_INSTALMENT_VERSION_sum,CC_NUM_INSTALMENT_NUMBER_min,CC_NUM_INSTALMENT_NUMBER_max,CC_NUM_INSTALMENT_NUMBER_mean,CC_NUM_INSTALMENT_NUMBER_std,CC_NUM_INSTALMENT_NUMBER_sum,CC_DAYS_INSTALMENT_min,CC_DAYS_INSTALMENT_max,CC_DAYS_INSTALMENT_mean,CC_DAYS_INSTALMENT_std,CC_DAYS_INSTALMENT_sum,CC_DAYS_ENTRY_PAYMENT_min,CC_DAYS_ENTRY_PAYMENT_max,CC_DAYS_ENTRY_PAYMENT_mean,CC_DAYS_ENTRY_PAYMENT_std,CC_DAYS_ENTRY_PAYMENT_sum,CC_AMT_INSTALMENT_min,CC_AMT_INSTALMENT_max,CC_AMT_INSTALMENT_mean,CC_AMT_INSTALMENT_std,CC_AMT_INSTALMENT_sum,CC_AMT_PAYMENT_min,CC_AMT_PAYMENT_max,CC_AMT_PAYMENT_mean,CC_AMT_PAYMENT_std,CC_AMT_PAYMENT_sum,CC_DAYS_DELAY_min,CC_DAYS_DELAY_max,CC_DAYS_DELAY_mean,CC_DAYS_DELAY_std,CC_DAYS_DELAY_sum,CC_PAID_OVER_AMOUNT_min,CC_PAID_OVER_AMOUNT_max,CC_PAID_OVER_AMOUNT_mean,CC_PAID_OVER_AMOUNT_std,CC_PAID_OVER_AMOUNT_sum,CC_PAID_OVER_min,CC_PAID_OVER_max,CC_PAID_OVER_mean,CC_PAID_OVER_std,CC_PAID_OVER_sum,CC_DPD_diff_min,CC_DPD_diff_max,CC_DPD_diff_mean,CC_DPD_diff_std,CC_DPD_diff_sum,CC_DBD_diff_min,CC_DBD_diff_max,CC_DBD_diff_mean,CC_DBD_diff_std,CC_DBD_diff_sum,CC_LATE_PAYMENT_min,CC_LATE_PAYMENT_max,CC_LATE_PAYMENT_mean,CC_LATE_PAYMENT_std,CC_LATE_PAYMENT_sum,CC_INSTALMENT_PAYMENT_RATIO_min,CC_INSTALMENT_PAYMENT_RATIO_max,CC_INSTALMENT_PAYMENT_RATIO_mean,CC_INSTALMENT_PAYMENT_RATIO_std,CC_INSTALMENT_PAYMENT_RATIO_sum,CC_LATE_PAYMENT_RATIO_min,CC_LATE_PAYMENT_RATIO_max,CC_LATE_PAYMENT_RATIO_mean,CC_LATE_PAYMENT_RATIO_std,CC_LATE_PAYMENT_RATIO_sum,CC_INS_IS_DPD_OVER_100_min,CC_INS_IS_DPD_OVER_100_max,CC_INS_IS_DPD_OVER_100_mean,CC_INS_IS_DPD_OVER_100_std,CC_INS_IS_DPD_OVER_100_sum,CC_DEFICIT_PCT_min,CC_DEFICIT_PCT_max,CC_DEFICIT_PCT_mean,CC_DEFICIT_PCT_std,CC_DEFICIT_PCT_sum,CC_SK_ID_PREV_nunique
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,100001,100001,100001.0,0.0,700007,1.0,2.0,1.142857,0.377964,8.0,1,4,2.714286,1.112697,19,1619.0,2916.0,2187.714286,653.989733,15314.0,1628.0,2916.0,2195.000000,643.904237,15365.0,3951.000,17397.900,5885.132143,5076.676624,41195.925,3951.000,17397.900,5885.132143,5076.676624,41195.925,-11.0,36.0,7.285714,14.625483,51.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,-11.0,36.0,7.285714,14.625483,51.0,-36.0,11.0,-7.285714,14.625483,-51.0,0,1,0.571429,0.534522,4,1.0,1.0,1.0,0.0,7.0,0.0,1.0,0.571429,0.534522,4.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,2
100002,100002,100002,100002.0,0.0,1900038,1.0,2.0,1.052632,0.229416,20.0,1,19,10.000000,5.627314,190,25.0,565.0,295.000000,168.819430,5605.0,49.0,587.0,315.421053,172.058877,5993.0,9251.775,53093.745,11559.247105,10058.037722,219625.695,9251.775,53093.745,11559.247105,10058.037722,219625.695,12.0,31.0,20.421053,4.925171,388.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,12.0,31.0,20.421053,4.925171,388.0,-31.0,-12.0,-20.421053,4.925171,-388.0,1,1,1.000000,0.000000,19,1.0,1.0,1.0,0.0,19.0,1.0,1.0,1.000000,0.000000,19.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,1
100003,100003,100003,100003.0,0.0,2500075,1.0,2.0,1.040000,0.200000,26.0,1,12,5.080000,3.134751,127,536.0,2310.0,1378.160000,757.453556,34454.0,544.0,2324.0,1385.320000,757.325432,34633.0,6662.970,560835.360,64754.586000,110542.592300,1618864.650,6662.970,560835.360,64754.586000,110542.592300,1618864.650,1.0,14.0,7.160000,3.726929,179.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0,1.0,14.0,7.160000,3.726929,179.0,-14.0,-1.0,-7.160000,3.726929,-179.0,1,1,1.000000,0.000000,25,1.0,1.0,1.0,0.0,25.0,1.0,1.0,1.000000,0.000000,25.0,0,0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,3
100004,100004,100004,100004.0,0.0,300012,1.0,2.0,1.333333,0.577350,4.0,1,3,2.000000,1.000000,6,724.0,784.0,754.000000,30.000000,2262.0,727.0,795.0,761.666667,34.019602,2285.0,5357.250,10573.965,7096.155000,3011.871810,21288.465,5357.250,10573.965,7096.155000,3011.871810,21288.465,3.0,11.0,7.666667,4.1633

In [135]:
# # Saving install_df to a csv to prevent repetitive rerun of the program
# install_df.to_csv("Data/install_df.csv")

In [136]:
def load_install_info(df, X_test, X_val=None, validation_set=False):

    #Loading csv file
    install_df = pd.read_csv("Data/install_df.csv")

    #Joining this new data and filling NAs with 0
    df = pd.merge(df, install_df, how='left', left_index=True, right_index=True).fillna(0)
    X_test = pd.merge(X_test, install_df, how='left', left_index=True, right_index=True).fillna(0)
    if validation_set:
        X_val = pd.merge(X_val, install_df, how='left', left_index=True, right_index=True).fillna(0)

## 4 Resampling our training dataset

As we've seen at the beginning of this part, our dataset has a very big imbalance with 92% of rows with the TARGET = 0 and only 8% with the Target variable equal to 1.

To reduce this imbalance, we will perform oversampling on our minority class.

Of course, **oversampling will only be performed on our train set**.

In [137]:
#Importing imblearn to be able to apply different kinds of oversampling
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SVMSMOTE
from imblearn.over_sampling import ADASYN

def over_sample_train(X_train, y_train, method: str):
    
    if method == 'SMOTE':
        #Importing the SMOTE algorithm with default values
        sm = SMOTE()

        #Generating our resampled dataset
        X_train_res, y_train_res = sm.fit_resample(df, y_train)
    
    elif method == 'SVMSMOTE':
        svm = SVMSMOTE()
        #Generating our resampled dataset
        X_train_res, y_train_res = svm.fit_resample(df, y_train)
        
    else:
        ada = ADASYN()
        #Generating our resampled dataset
        X_train_res, y_train_res = svm.fit_resample(df, y_train)
    
    return X_train_res, y_train_res, X_train, y_train
        

# print(X_train_res.shape)
# print(y_train_res.value_counts())
# #We have successfully removed the imbalance from our dataset and equalized the number of observations for each class

In [ ]:
# #Renaming the resampled variables for ease of use
# X_train_initial = df.copy()
# y_train_initial = y_train

# X_train = X_train_res.copy()
# y_train = y_train_res

In [ ]:
# #Deleting some variables to clear memory
# import sys
# def sizeof_fmt(num, suffix='B'):
#     for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
#         if abs(num) < 1024.0:
#             return "%3.1f %s%s" % (num, unit, suffix)
#         num /= 1024.0
#     return "%.1f %s%s" % (num, 'Yi', suffix)

# for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
#                          key= lambda x: -x[1])[:20]:
#     print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

# del df, prev_app, bureau_balance, cash, install, cc, bureau, X

Now that we have resampled our dataset, we want to perform **feature selection** to reduce the number of features and prevent overfitting.

## 6. Feature Selection

### 6.1 Removing low variance features

In [139]:
# Perform feature selection using a variance threshold
from sklearn.feature_selection import VarianceThreshold

def perform_variance_selection(X_train, X_test, X_val=None, validation_set=False, threshold=(0.02)):
    #We select 2% as our variance threshold, but this is a hyperparameter that we will be able to optimize later
    sel = VarianceThreshold(threshold=threshold)
    sel.fit(X_train)
    print("Initial shape:", X_train.shape)
    #Using our selector to remove columns from our 3 sets
    X_train = sel.transform(X_train)
    X_test = sel.transform(X_test)
    if validation_set:
        X_val = sel.transform(X_val)

    #Creating a list of encoded columns to preserve their names
    i = 0
    #Retrieving the boolean values for each column (is the column kept or not)
    boolean_cols = sel.get_support()
    encoded_cols = []
    initial_cols = df.columns
    for i in range(len(initial_cols)):
        if boolean_cols[i] == True:
            encoded_cols.append(initial_cols[i])
        i += 1

    #The selector has transformed our dataframes into np array, let's turn them back into a DataFrame
    X_train = pd.DataFrame(X_train, columns=encoded_cols)
    X_test = pd.DataFrame(X_test, columns=encoded_cols)
    
    if validation_set:
        X_val = pd.DataFrame(X_val, columns=encoded_cols)
        
    print("Final shape: ",X_train.shape)

### 6.2 Removing highly correlated features

In [140]:
# Function to list features that are correlated
# Adds the first of the correlated pair only (not both)
def correlatedFeatures(dataset, threshold):
    correlated_columns = set()
    correlations = dataset.corr()
    for i in range(len(correlations)):
        for j in range(i):
            if abs(correlations.iloc[i,j]) > threshold:
                correlated_columns.add(correlations.columns[i])
    return correlated_columns

# cf = correlatedFeatures(X_train, 0.85)
# cf

In [ ]:
# #Removing our highly correlated features
# X_train = X_train.drop(cf, axis=1)
# X_test = X_test.drop(cf, axis=1)
# X_val = X_val.drop(cf, axis=1)

# print(X_train.shape, X_test.shape, X_val.shape)

### 6.3 Selecting best features

We will now use the Kbest algorithm to select the X best features

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression

kbest = SelectKBest(score_func=f_regression, k=15)
kbest.fit(X_train, y_train)

print("Selected features:", list(X_train.columns[kbest.get_support()]))

In [ ]:
# X_train_sel = kbest.transform(X_train)
# X_val_sel = kbest.transform(X_val)
# X_test_sel = kbest.transform(X_test)

Since we are **mostly interested in precision** (we do not want to avoid bad borrowers at any cost, since we still need to make money by allowing the largest part possible of good lenders), we shoud not remove features.



## 6. Model training

In this phase, I will train and compare 2 different models :

- A **Logistic regression model**
- A **Support Vector Machine / Classifier**

We will include the scaling of our data in a pipeline within each model.

### 6.1 Selecting a Performance Metric

Our task is to try to detect as many "bad borrowers" as possible while avoiding false negatives and losing too many clients.
It is hard to select the right metric without knowing 

### 6.1 Logistic Regression Model

In [ ]:
# #Looking at the best number of features for Logistic Regression
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import StandardScaler
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import precision_score

# scores = []
# for k in range(1,99):
#     #We need to scale the dataset before applying Logistic Regression because sklearn log_r includes L2 regularization
#     kbest = SelectKBest(score_func=f_regression, k=k)
#     pipe_lr = Pipeline([('scaler', StandardScaler()), ('kbest', kbest), ('log_r', LogisticRegression(max_iter = 1000))])
    
#     pipe_lr.fit(X_train, y_train)

#     train_predictions = pipe_lr.predict(X_train)
#     val_predictions = pipe_lr.predict(X_val)
#     roc_auc_train = roc_auc_score(y_train, train_predictions)
#     roc_auc_val = roc_auc_score(y_val, val_predictions)
#     mean_roc = (roc_auc_train + roc_auc_val)/2
#     preci_val = precision_score(y_val, val_predictions)
    
#     scores.append({'k': k, 'roc_train': roc_auc_train, 'roc_val': roc_auc_val,
#                    'mean_roc': mean_roc, 'precision': preci_val})

# scores = pd.DataFrame(scores)

# scores

In [ ]:
# sns.lineplot(data=scores, x='k', y='roc_train', color='green')
# sns.lineplot(data=scores, x='k', y='roc_val', color='blue')
# ax = sns.lineplot(data=scores, x='k', y='mean_roc', color='red')
# ax.set_xlim(left=0, right=20)
# plt.show()

In [ ]:
# sns.lineplot(data=scores, x='k', y='precision', color='red')
# plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, f1_score
from sklearn.metrics import roc_auc_score

#We need to scale the dataset before applying Logistic Regression because sklearn log_r includes L2 regularization
pipe_lr = Pipeline([('scaler', StandardScaler()), 
                    ('log_r', LogisticRegression(max_iter = 3000))])

pipe_lr.fit(X_train, y_train)

train_predictions = pipe_lr.predict(X_train)
test_predictions = pipe_lr.predict(X_test)

print("Logistic Regression results")
print("TRAIN:")
print(classification_report(y_train, train_predictions))
print("ROC AUC train : {:.2f}".format(roc_auc_score(y_train, train_predictions)))
print("----------------------")
print("TEST:")
print(classification_report(y_test, test_predictions))
print("ROC AUC train : {:.2f}".format(roc_auc_score(y_test, test_predictions)))

In [ ]:
print(X_test.shape, y_test.shape)

### 6.2 Support Vector Classification (Linear)

In [ ]:
from sklearn.svm import LinearSVC

pipe_lsvc = Pipeline([('scaler', StandardScaler()), ('svc_l', LinearSVC())])

pipe_lsvc.fit(X_train, y_train)

train_predictions = pipe_lsvc.predict(X_train)
test_predictions = pipe_lsvc.predict(X_test)

print("Logistic Regression results")
print("TRAIN:")
print(classification_report(y_train, train_predictions))
print("----------------------")
print("TEST:")
print(classification_report(y_test, test_predictions))

### 6.3 KNeighbors Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

pipe_knc = Pipeline([('scaler', StandardScaler()), ('knc', KNeighborsClassifier())])

pipe_knc.fit(X_train, y_train)

train_predictions = pipe_knc.predict(X_train)
test_predictions = pipe_knc.predict(X_test)

print("Logistic Regression results")
print("TRAIN:")
print(classification_report(y_train, train_predictions))
print("----------------------")
print("TEST:")
print(classification_report(y_test, test_predictions))

### 6.4 Support Vector Classification

In [ ]:
from sklearn.svm import SVC

#pipe_svc = Pipeline([('scaler', StandardScaler()), ('svc', SVC(verbose=2))])

#pipe_svc.fit(X_train, y_train)

#train_predictions = pipe_svc.predict(X_train)
#test_predictions = pipe_svc.predict(X_test)

print("Logistic Regression results")
print("TRAIN:")
#print(classification_report(y_train, train_predictions))
print("----------------------")
print("TEST:")
#print(classification_report(y_test, test_predictions))

### 6.5 Ensemble Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

pipe_gbc = Pipeline([('scaler', StandardScaler()), ('EGBC', GradientBoostingClassifier(verbose=3))])

pipe_gbc.fit(X_train, y_train)

train_predictions = pipe_gbc.predict(X_train)
test_predictions = pipe_gbc.predict(X_test)

print("Logistic Regression results")
print("TRAIN:")
print(classification_report(y_train, train_predictions))
print("----------------------")
print("TEST:")
print(classification_report(y_test, test_predictions))